# Previsioni delle quantità di CO<sub>2</sub> emesse da veicoli

## Caricamento dei dati
Per questo progetto sono stati utilizzati tre dataset differenti. <br>
Quindi sono necessarie tre esplorazioni separate dei dati e un'omogeneizzazione dei dati, al fine di ottenere un unico dataset per l'addestramento. <br>

In [ ]:
!pip install pykan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

from sklearn.model_selection import KFold, ParameterSampler, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    mean_squared_error, r2_score,
    mean_absolute_error, max_error
)
import sklearn.metrics as metrics
from scipy import stats
import xgboost as xgb

import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import copy
import types
import seaborn as sns
sns.set_theme()
from kan import *
import random
import inspect
import json
import os.path
from urllib.request import urlretrieve

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

In [ ]:
DATASETS = {
    "car_emissions_spain2022.csv": "https://raw.githubusercontent.com/vMxster/Thesis/main/Datasets/car_emissions_spain2022.csv",
    "car_emissions_canada2014.csv": "https://raw.githubusercontent.com/vMxster/Thesis/main/Datasets/car_emissions_canada2014.csv",
    "car_emissions_uk2013.csv": "https://raw.githubusercontent.com/vMxster/Thesis/main/Datasets/car_emissions_uk2013.csv"
}

if IN_COLAB:
    print("Running in Google Colab. Downloading datasets to the current directory.")
    for filename, dataset_url in DATASETS.items():
        if not os.path.exists(filename):
            urlretrieve(dataset_url, filename)
else:
    print("Not in Google Colab. Downloading datasets to the 'Datasets' subdirectory.")
    DATASETS_DIR = "datasets_car_emissions"
    if not os.path.exists(DATASETS_DIR):
        os.makedirs(DATASETS_DIR)

    for filename, dataset_url in DATASETS.items():
        filepath = os.path.join(DATASETS_DIR, filename)
        if not os.path.exists(filepath):
            urlretrieve(dataset_url, filepath)

In [ ]:
if IN_COLAB:
    spain_emissions = pd.read_csv("car_emissions_spain2022.csv")
else:
    spain_emissions = pd.read_csv(os.path.join(DATASETS_DIR, "car_emissions_spain2022.csv"))
spain_emissions.head(5)

In [ ]:
if IN_COLAB:
    canada_emissions = pd.read_csv("car_emissions_canada2014.csv")
else:
    canada_emissions = pd.read_csv(os.path.join(DATASETS_DIR, "car_emissions_canada2014.csv"))
canada_emissions.head(5)

In [ ]:
if IN_COLAB:
    uk_emissions = pd.read_csv("car_emissions_uk2013.csv", low_memory=False)
else:
    uk_emissions = pd.read_csv(os.path.join(DATASETS_DIR, "car_emissions_uk2013.csv"), low_memory=False)
uk_emissions.head(5)

# **Esplorazione dei singoli dataset**

## Esplorazione del dataset `uk_emissions`

Il dataset in questione è stato reso fruibile dalla Vehicle Certification Agency (VCA) del Department for Transport britannico, al fine di reperire informazioni inerenti a tutte le vetture nuove in vendita nel Regno Unito nel lasso temporale 2000-2013, e quelle usate immatricolate per la prima volta a partire dal 1° marzo 2001. Il termine *nuova* si riferisce ad una qualsiasi automobile allora disponibile per l'acquisto o il leasing presso un concessionario e che non fosse stata precedentemente registrata. Sempre citando la [documentazione](https://www.gov.uk/co2-and-vehicle-tax-tools), abbiamo che entrambi i veicoli nuovi ed usati possono essere cercati per trovarne i corrispondenti:
- Fuel consumption and CO<sub>2</sub> emissions (by make and model)
- vehicle tax information (by make, model, registration date and current tax tables)
- The cost of tax for all vehicle types

Mentre sono disponibili solamente per le macchine nuove le seguenti informazioni:

- Tax band, including Band A (exempt from tax)
- Fuel economy
- Annual fuel running costs
- Company car taxation, based on CO<sub>2</sub> bands
- Alternative fuel types

Potremmo quindi inziare l'analisi del dataset verificando la veridicità di quest'ultimo ragguaglio, ossia controllando che le auto usate assumano valore NaN in corrispondenza delle colonne sopracitate. Sfortunatamente non disponiamo di una variabile che distingua esplicitamente i mezzi nuovi da quelli usati, ma sappiamo che le osservazioni con `date_of_change` (assumendo si riferisca alla data del passaggio di proprietà) diverso da NaN indicano una vettura sicuramente di seconda mano. Quindi sebbene queste rappresentino un sottoinsieme delle usate (mancherebbero quelle usate da un unico proprietario) eseguiamo il test:

In [ ]:
used_cars_subset = uk_emissions.loc[~uk_emissions["date_of_change"].isna()]
columns_supposedly_nan = used_cars_subset[["tax_band",
                                           "standard_12_months",
                                           "standard_6_months",
                                           "first_year_12_months",
                                           "first_year_6_months",
                                           "fuel_cost_12000_miles"]]
print(columns_supposedly_nan)
columns_supposedly_nan.isna().all()

Abbiamo appurato l'attendibilità della documentazione riguardo al valore degli attributi supposti NaN nelle auto usate. Ci rimane da esaminare l'assenza di tipi di carburante alternativi (contraddistinti dal valore _Petrol/E85(Flex Fuel)_) nella feature categorica `fuel_type` in tali osservazioni:

In [ ]:
used_cars_subset["fuel_type"].isin(["Petrol / E85 (Flex Fuel)"]).any()

Possiamo ritenerci soddisfatti. Sarebbe ora lecito chiedersi quali siano i marchi automobilistici più popolari nel dataset. Soddisfiamo la nostra curiosità visualizzando i primi dieci in ordine decrescente per numero di comparse in un diagramma a torta, condensando i restanti brand nello spicchio `Others` per motivi di comprensibilità.

In [ ]:
# n.b., i termini "manufacturer" e "brand" sono utilizzati come sinonimi
print("[" + str(uk_emissions["manufacturer"].nunique()) \
      + "] different manufacturers and ["               \
      + str(uk_emissions["manufacturer"].count())       \
      + "] total cars are registered")
manufacturers = uk_emissions["manufacturer"].value_counts()
majority_brands = manufacturers[:10]
minority_brands = manufacturers[10:]
# oppure raggruppando i brand che rappresentano meno del 5%
# minority_brands_perc = manufacturers[manufacturers < uk_emissions["manufacturer"].count() * 5 / 100]
majority_brands["Others"] = minority_brands.sum()
my_explode = np.append(np.zeros(majority_brands.count() - 1), 0.1)
majority_brands.plot.pie(title = "TOP 10 BRANDS", explode = my_explode, shadow = True, autopct = "%.2f%%", figsize=(6, 6));

In [ ]:
print("OTHERS: 11th to 62nd brand in descending order of popularity:\n")
percentages = minority_brands.to_numpy() * 100 / uk_emissions["manufacturer"].count()
list_of_series = [minority_brands.index.values, minority_brands.values, percentages]
minority_df = pd.DataFrame({"manufacturer"  : minority_brands.index.values,
                            "# occurrences" : minority_brands.values,
                            "percentage"    : percentages})
n_brands = uk_emissions["manufacturer"].nunique()
minority_df.index = range(11, n_brands + 1)
print(minority_df)

Vediamo ora all'interno di ciascuno dei brand graficati in precedenza quale sia il modello con più occorrenze registrate, servendoci di un istogramma dove ci cureremo di mantenere l'ordine dei produttori secondo le percentuali dianzi calcolate, per una maggiore leggibilità.

In [ ]:
brands_and_models = uk_emissions[["manufacturer", "model"]].copy()
brands_and_models = brands_and_models[brands_and_models.manufacturer.isin(majority_brands.index)]
print("Manufacturers left after screening: [" + str(brands_and_models.manufacturer.nunique()) + "]")
majority_brands = majority_brands.drop("Others")
brands_and_models = brands_and_models.value_counts(["manufacturer", "model"]) \
        .reset_index(name = "count")                                          \
        .sort_values(["count"], ascending = False)                            \
        .drop_duplicates(["manufacturer"])                                    \
        .set_index(["manufacturer"])                                          \
        .reindex(majority_brands.index) # ri-ordiniamo secondo la classifica dei brand stilata in precedenza
brands_and_models["car"] = brands_and_models.index + "\n" + brands_and_models["model"] # concateniamo le stringhe
brands_and_models.plot.bar(title = "MOST POPULAR MODEL FOR EACH OF THE TOP 10 BRANDS ", x = "car", y = "count");

Rimarcherei come un marchio automobilistico possa ripetersi più volte nel dataframe per via degli ipotetici diversi modelli ad esso associati, mentre le molteplici apparizioni di uno stesso modello sono giustificate da una o più differenze di valori in variabili attinenti a specifiche tecniche quali la `engine_capacity` o il `fuel_type`. Potremmo infatti trovare, e.g., una _Ford Courier_ sia a benzina (petrol) con cilindrata (engine_capacity) di 1299 cm<sup>3</sup> che a diesel con cilndrata di 1753 cm<sup>3</sup>. Andiamo ad estrarre qualche insight, facendo attenzione a non assumere che non vi possano essere modelli omonimi tra brand diversi (contando quindi i valori unici in coppia con il relativo brand). Di seguito ignoreremo le le differenze di trasmissione.

In [ ]:
n_cars = len(uk_emissions[["manufacturer", "model", "description"]].copy().drop_duplicates())
n_models = len(uk_emissions[["manufacturer", "model"]].copy().drop_duplicates())
n_brands = len(uk_emissions[["manufacturer"]].copy().drop_duplicates())
print("Average number of engine variants per model: [{:.2f}]".format(n_cars / n_models))
print("Average number of models per manufacturer:  [{:.2f}]".format(n_models / n_brands))

Indaghiamo la relazione tra la classi dello [Standard europeo sulle emissioni inquinanti](https://en.wikipedia.org/wiki/European_emission_standards) e lo sprigionamento di CO delle vetture, per mezzo di un diagramma a scatola e baffi. Osserviamo come le medie dei box siano ordinate. La presenza di outliers, assieme al fatto che uno stesso range di valori di monossido di carbonio possa appartenere a più classi, sono chiari indicatori di come il CO non sia, ragionevolmente, l'unico inquinante tenuto in considerazione per determinare l'appartenenza ad una certa categoria `euro_standard`. Com'è infatti possibile verificare nella pagina a cui rinvia l'hyperlink, THC, NMHC, NH<sub>3</sub> e NO<sub>x</sub> sono alcuni tra gli altri composti valutati.

In [ ]:
quartiles_first = uk_emissions.boxplot(column = "co_emissions", by = "euro_standard", showmeans = True);
quartiles_first.set_ylabel("co_emissions");
quartiles_first.set_title("");
plt.ylim(0, 2500);
plt.suptitle("");

Similmente a quanto fatto sopra andiamo, attraverso un diagramma degli estremi e dei quartili, ad analizzare l'eventuale legame tra `tax_band` e `co2_emissions`. Questa volta notiamo tuttavia come il valore massimo di diossido di carbonio (in g/km) di una certa fascia corrisponda al minimo di quella successiva. In effetti, come riportato nella [guida](https://assets.publishing.service.gov.uk/media/6603f64b13397a0011e419be/v149-rates-of-vehicle-tax-for-cars-motorcycles-light-goods-vehicles-and-private-light-goods-vehicles.pdf) redatta dalla Driver and Vehicle Licensing Agency (DVLA), la CO<sub>2</sub> risulterebbe essere, assieme al `fuel_type`, l'unico elemento considerato nel decretare gli scaglioni di tasse.

In [ ]:
quartiles_second = uk_emissions.boxplot(column = "co2", by = "tax_band", showmeans=True, figsize = (10, 16));
quartiles_second.set_ylabel("co2_emissions");
quartiles_second.set_title("");
plt.suptitle("");

Concludiamo l'esplorazione di questo dataset con un grafico a dispersione che leghi l'`engine_capacity` (volume del motore in cm<sup>3</sup>) di ciascuna vettura con i relativi `combined_metric` (consumi in l/100km). Come ci aspettavamo, in linea di massima, al crescere del primo aumenta anche il secondo ed il trend potrebbe, come suggerisce anche l'indice di correlazione di Pearson tra le due colonne, essere approssimato in maniera soddisfacente da una retta.

In [ ]:
uk_emissions.plot.scatter("engine_capacity", "combined_metric", s=7, c="red");
print("Pearson correlation coefficient: [{:.2f}]".format(uk_emissions["engine_capacity"].corr(uk_emissions["combined_metric"])))

## Esplorazione del dataset `spain_emissions`
### Rimozione delle righe riguardanti veicoli elettrici

All'interno del dataset proveniente dal ministero spagnolo sono presenti anche dati riguardanti macchine elettriche. Questa tipologia di dato non è interessante per l'obiettivo del modello, ovvero la quantità di CO<sub>2</sub> emessa.

La cella seguente analizza i valori delle righe che hanno `engine_type` "Eléctricos puros", estraendo solo le colonne sul minimo e massimo di consumo di litri di carburante e sul minimo e massimo di emissioni di CO<sub>2</sub> per km percorso.

In [ ]:
columns = ["emissions_min_gCO2_km", "emissions_max_gCO2_km"]
spain_emissions.loc[spain_emissions["engine_type"] == "Eléctricos puros", columns].isna().all()

Come era previdibile, tutti i valori delle emissioni di CO<sub>2</sub> per km nelle righe riguardanti i veicoli elettrici sono mancanti, quindi le rispettive righe possono essere eliminate, poichè non utili per l'addestramento del modello, il cui obiettivo è la previsione dell'emissione di CO<sub>2</sub> per veicoli a motore termico.

In [ ]:
electric_cars_index = spain_emissions.loc[spain_emissions["engine_type"] == "Eléctricos puros"].index
spain_emissions = spain_emissions.drop(electric_cars_index)
spain_emissions["engine_type"].unique()


Come si può notare dall'output della cella precedente, la colonna `engine_type` non contiene il valore "Eléctricos puros".

### Rimozione delle colonne contenenti solo valori `NaN`
Nel dataset dell'emissioni di CO<sub>2</sub> proveniente dalla Spagna la colonne `type_hybrid`, `electric_consumption_kwh_100km`, `battery_capacity_kwh ` contengono solo valori `NaN`.

In [ ]:
all_nan_columns = ["type_hybrid", "electric_consumption_kwh_100km", "battery_capacity_kwh"]
spain_emissions[all_nan_columns].isna().values.all()

In [ ]:
spain_emissions = spain_emissions.drop(all_nan_columns, axis=1)

### Rimozione delle righe senza un valore valido di trasmissione
La colonna `transmission` contiene il tipo di trasmissione del veicolo, quindi la modalità di cambio della marcia, se automatica o manuale, che nella colonna sono rispettivamente "A" e "M". L'output della prossima cella mostra che sono presenti 35 righe con valori di trasmissioni non validi, poichè nel dataset spagnolo "SC" sta per "Sin clasificasion", ovvero non classificato. Perciò le righe con valori di trasmissione non validi, essendo un numero molto ridotto, verrano eliminate.

In [ ]:
spain_emissions["transmission"].value_counts()

In [ ]:
valid_transmissions = spain_emissions["transmission"].isin(["A", "M"])
spain_emissions["transmission"] =  spain_emissions.loc[valid_transmissions, "transmission"]

## Esplorazione del dataset `canada_emissions`

### Gestione dimensioni motore

All'interno del dataset proveniente dal ministero canadese sono presenti le dimensioni in L (Litri) del motore dei vari veicoli. Decidiamo di trasformarle in cm$^3$ rinominando quindi la feature Engine_cm3.

In [ ]:
canada_emissions = canada_emissions.rename(columns={"Engine size (L)": "Engine_cm3"})
canada_emissions["Engine_cm3"] = canada_emissions["Engine_cm3"] * 1000
canada_emissions.head(5)

### Trattamento valori mancanti

Con la cella seguente verifico se il dataset contiene dei dati mancanti (nan).

In [ ]:
canada_emissions.isna().values.any()

Visto che il risultato è false non c'è alcun valore mancante da gestire.

### Grafici
Questa sezione contiene alcuni grafici che evidenziano alcuni aspetti del dataframe `canada_emissions`.

L'istogramma seguente mostra la distribuzione delle emissioni di CO$_2$ nel dataframe canadese. I valori si concentrano principalmente nell'intervallo [150,350].

In [ ]:
canada_emissions["CO2 emissions (g/km)"].plot.hist(bins=20,title="CO\u2082 emissions distribution");

L'istogramma seguente mostra le dimensioni dei motori dei vari veicoli nel dataframe canadese. I valori si concentrano principalmente nell'intervallo [1500,6000].

In [ ]:
canada_emissions["Engine_cm3"].plot.hist(bins=20,title="Engine size");

Il seguente grafico a torta mostra il numero di cilindri (in percentuale) dei vari veicoli nel dataframe canadese. La maggior parte di essi ha 6 cilindri.

In [ ]:
canada_emissions["Cylinders"].value_counts().plot.pie(title="Cylinders number" , shadow=True, figsize=(6,6), labels=None)
labels = [str(cylinder) + " cylinders" for cylinder in canada_emissions["Cylinders"].value_counts().index]
plt.legend(labels);

Il seguente grafico a barre mostra i vari tipi di veicoli presenti nel dataset canadese con la relativa frequenza.

In [ ]:
canada_vehicle_class = canada_emissions["Vehicle class"].value_counts()
plt.bar(canada_vehicle_class.index, canada_vehicle_class.values)
plt.xticks(rotation='vertical')
plt.title("Vehicle class")
plt.show()

# **Omogeneizzazione ed unione dei dati**
Avendo reperito tre set di dati ciascuno da una fonte differente, sebbene questi concernano il medesimo dominio e condividano pertanto una considerevole quantità di attributi, potremmo essere erroneamente portati a compiere un troncamento delle colonne, conservando esclusivamente quelle appartenenti all'intersezione delle tre tabelle, tuttavia così facendo andremmo a trascurare il caso-limite in cui le features eliminate dovessero essere le uniche legate, attraverso un relazione non randomica, a quella indagata. Una soluzione più scrupolosa consisterebbe, al contrario, nell'**unione** degli **attributi**, colmando opportunamente le celle vuote formatesi. In effetti, come visto a lezione, i valori NAN possono essere sostituiti, tra gli altri, da (a seconda che le colonne siano di tipo categorico o numerico):
- media
- moda
- mediana

Questo ragionamento va ad ogni modo applicato nei limiti del ragionevole: se una variabile dovesse essere presente solamente in uno dei tre dataset, sarebbe piuttosto fuorviante "inventare" i 2/3  dei valori da essa assunti (ipotizzando i dataset di egual dimensione). Decidiamo quindi di mantenere le colonne presenti in almeno due dataset su tre. In aggiunta, le stesse variabili in comune, malgrado facciano riferimento allo stesso concetto, si manifestano saltuariamente sotto denominazioni diverse, ed a loro volta i valori da queste assunti potrebbero avere formato dissimile (e.g. capitalizzazione delle lettere nel caso testuale o numero di cifre significative in quello numerico).







## Omogeneizzazione di `uk_emissions`

Rimuoviamo le colonne proprie di questo dataset, per moderare la presenza di NaN o dati sostitutivi in quello risultante dall'unione:

In [ ]:
uk_emissions = uk_emissions.drop(columns = ["file", "description", "euro_standard", "tax_band", "transmission", "urban_metric",
                                            "extra_urban_metric", "urban_imperial", "extra_urban_imperial", "combined_imperial",
                                            "noise_level", "thc_emissions", "co_emissions", "nox_emissions", "thc_nox_emissions",
                                            "particulates_emissions", "fuel_cost_12000_miles", "fuel_cost_6000_miles", "standard_12_months",
                                            "standard_6_months", "first_year_12_months", "first_year_6_months", "date_of_change"])

Trasformiamo i nomi delle variabili in quelli concordati con gli altri componenti del gruppo:

In [ ]:
uk_emissions.columns = uk_emissions.columns.str.capitalize()
uk_emissions.rename(columns = {"Engine_capacity" : "Engine_cm3",
                               "Combined_metric" : "Fuel_consumption",
                               "Co2" : "CO2_Emissions"}, inplace = True)

Ci troviamo ora dinnanzi ad una delle principali complicazioni indotte dall'utilizzo di più dataset. L'obbiettivo sarebbe quello di manipolare i nomi delle vetture in modo tale da garantire la consistenza semantica del dataset finale: non vorremmo che, e.g., le automobili _BMW 3 Series_ e _Bmw Series 3_ siano considerate differenti, perlomeno sugli attributi `Manufacturer`e `Model`, anche per consentire all'algoritmo di machine learning di trarre conclusioni rispetto al modello specifico. Il problema non si pone per variabili numeriche come i consumi o la cilindrata, dove siamo interessati più al range di appartenenza piuttosto che ai valori precisi. Vista la mancanza di un pattern universale e siccome risulterebbe eccessivamente gravoso indagare tutte le ~90'000 osservazioni, ci limiteremo ad uniformare quantomeno un campione di automobili analizzato.

In [ ]:
uk_emissions.dropna(inplace = True)
uk_emissions["Model"] = uk_emissions["Model"].replace("[\(\[].*?[\)\]]", "", regex = True)
uk_emissions["Model"] = uk_emissions.apply(lambda row : row["Model"].replace(str(row["Manufacturer"]), ''), axis=1)
uk_emissions["Model"] = uk_emissions["Model"].str.split(" ").str[0].str.replace(",", "")
uk_emissions

## Omogeneizzazione di `spain_emissions`

### Traduzione spagnolo-inglese
Il dataset proveniente dalla Spagna contiene due colonne in spagnolo, `engine_type` e `market_segment`. Quindi è necessaria una traduzione dallo spagnolo all'inglese per entrambe le colonne, così da adattarle agli altri due dataset.

Definisco una funzione per sostituire i valori di una colonna di un dataframe con dei nuovi valori passati come argomento, in questo caso la rispettiva traduzione in inglese, tramite una funzione di mapping fornita da pandas.

In [ ]:
def get_column_mapped_values(column, new_values):
    mapping = dict(zip(column.unique(), new_values))
    return column.map(mapping)

In [ ]:
#Traduzione della colonna engine_type
spanish_engine_type = spain_emissions["engine_type"].unique()
english_engine_type = ["Petrol", "Diesel", "Plug-in hybrid", "Petrol hybrid", "Natural gas",
                       "Diesel Hybrid", "Liquefied petroleum gas(LPG)", "Fuel cell", "Extended range"]

spain_emissions["engine_type"] = get_column_mapped_values(spain_emissions["engine_type"], english_engine_type)

Per la traduzione della colonna `market_segment` si fa riferimento al dataset canadese, il quale contiene una colonna per la classificazione dei veicoli, `Vehicle class`.


In [ ]:
canada_emissions["Vehicle class"].unique()

In [ ]:
#Traduzione della colonna market_segment
english_market_segment = ["Minicompact", "Compact", "Small off-road", "Mid-size", "Sport utility vehicle", "Full-size",
                          "Mid-size off-road", "Full-size off-road", "Minivan", "Luxury", "Minivan", "Van: Passenger"] \
                        + (["Van: Cargo"] * 2) + (["Lorry"] * 7)

spain_emissions["market_segment"] = get_column_mapped_values(spain_emissions["market_segment"], english_market_segment)

### Calcolo medie del consumo di carburante e delle emissioni di CO<sub>2</sub>
Sia per il consumo di carburante che per le emissioni di CO<sub>2</sub> sono presenti i valori di minimo, di massimo e la media
WLTP (<i>Worldwide harmonized Light vehicles Test Procedure</i>), ovvero le misure ottenute dal test standard a livello mondiale per il controllo delle emissioni e dei consumi dei veicoli.
Da questi tre valori ne verrà calcolata la media così da avere una sola colonna per i consumi di carburante e una sola colonna per i consumi di CO<sub>2</sub>, la variabile target da predire.

Prima di poter precedere è necessario controllare che in tutte le righe del dataframe sia presente almeno uno dei tre valori richiesti e che sia diverso da 0, sia per calcolare la media del consumo che delle emissioni.
<br>Nella cella successiva si può vedere esattamente in quante righe sono mancanti questi valori necessari.

In [ ]:
def isnaOrIsZero(dataframe):
    return dataframe.isna() | dataframe.eq(0)

In [ ]:
consumption_columns = ["consumption_min_l_100km", "consumption_max_l_100km", "avg_wltp_consumption_l_100km"]
emissions_columns = ["emissions_min_gCO2_km", "emissions_max_gCO2_km", "avg_wltp_emissions_gCO2_km"]

rows_no_consumption = isnaOrIsZero(spain_emissions[consumption_columns]).all(axis=1)
number_rows_missing_consumption = rows_no_consumption.sum()
print("Rows with no consumption_l_100km:", number_rows_missing_consumption)

rows_no_emissions = isnaOrIsZero(spain_emissions[emissions_columns]).all(axis=1)
number_rows_missing_emissions = rows_no_emissions.sum()
print("Rows with no emissions_gCO2_km:", number_rows_missing_emissions)

rows_no_emissions_consumptions = (rows_no_consumption & rows_no_emissions)
number_rows_missing_emissions_consumptions = rows_no_emissions_consumptions.sum()
print("Rows with no emissions_gCO2_km and no consumption_l_km:", number_rows_missing_emissions_consumptions)

Come riportato nell'output della cella precedente, ci sono <strong>1560</strong> righe senza alcun valore di consumo di carburante, <strong>1562</strong> righe senza alcun valore di emissioni di CO<sub>2</sub> e <strong>1560</strong> in cui è assente sia il consumo di carburante che le emissioni di CO<sub>2</sub>. <br>
Nella cella successiva vengono calcolate le medie di carburante ed emissioni di ciascuna riga e vengono inserite in due nuove colonne `consumption_l_100km`, `emissions_gCO2_km`. <br>

In [ ]:
mean_consumption = "consumption_l_100km"
mean_emissions = "emissions_gCO2_km"
spain_emissions[mean_consumption] = spain_emissions[consumption_columns].mean(axis=1)
spain_emissions[mean_emissions] = spain_emissions[emissions_columns].mean(axis=1)

In [ ]:
spain_emissions = spain_emissions.drop(consumption_columns + emissions_columns, axis=1)

Quindi vengono sostituiti i valori mancanti e i valori posti a 0, poichè considerati anch'essi mancanti, siccome le categorie di veicoli considerate dovrebbero presentare dei consumi di carburante e delle emissioni di CO<sub>2</sub> positivi. <br>
La cella successiva mostra le categorie di veicoli in cui sono assenti sia i consumi di carburante che le emissioni. Le due categorie con più occorrenze sono i "Van: Cargo" e "Lorry", due mezzi molto pesanti e con valori differenti rispetto ad altre categorie, quindi sarebbe più corretto riempire i valori assenti o posti a 0, con i mediani delle due rispettive categorie. Mentre per le restanti categorie si può utilizzare direttamente il mediano di tutta la colonna.

In [ ]:
most_missing_market_segments = spain_emissions.loc[rows_no_emissions_consumptions, "market_segment"].value_counts()[:5]
most_missing_market_segments

Le celle successive evidenziano la necessità di usare, come valore per riempire i valori NaN o posti a 0 delle categorie di veicoli "Van: Cargo" e "Lorry", i rispettivi mediani, anzichè usare direttamente il mediano di tutta la colonna, vista l'ampia differenza causata dalla presenza di veicoli di diverse dimensioni e consumi.

In [ ]:
columns = [mean_consumption, mean_emissions]
market_segments = ["Van: Cargo", "Lorry"]

In [ ]:
all_median = pd.DataFrame(spain_emissions[columns].median(), columns = ["All"]).T
van_cargo_lorries = spain_emissions.loc[spain_emissions["market_segment"].isin(market_segments), ["market_segment"] + columns]
van_cargo_lorries_median = van_cargo_lorries[van_cargo_lorries.ne(0)].groupby("market_segment").median()

In [ ]:
pd.concat([all_median, van_cargo_lorries_median]).style.set_caption("Market segments medians")

In [ ]:
def fillnaZeroes(market_segment, columns):
    vehicles = spain_emissions.loc[spain_emissions["market_segment"] == market_segment, columns].replace(0, np.nan)
    vehicles = vehicles.fillna(vehicles.median())
    return vehicles

In [ ]:
for market_segment in market_segments:
    spain_emissions.loc[spain_emissions["market_segment"] == market_segment, columns] = fillnaZeroes(market_segment, columns)

In [ ]:
other_market_segments = set(spain_emissions["market_segment"].unique()) - set(market_segments)
spain_emissions[columns] = spain_emissions[columns].replace(0, np.nan).fillna(spain_emissions[columns].median())

### Utilizzo di valori espliciti per il tipo di trasmissione
Il dataset spagnolo per indicare la tipologia di trasmissione utilizza solamente "A" e "M". Per adattarlo agli altri dataset e per renderlo più chiaro si è deciso di mappare i rispettivi valori con la versione estesa, ovvero "Automatic", "Manual".

In [ ]:
renaming_dict = {"A": "Automatic", "M": "Manual"}
spain_emissions["transmission"] = spain_emissions["transmission"].map(renaming_dict)

### Inserimento dell'anno del modello del veicolo
I dataset canadesi e britannici presentano entrambi una colonna per l'anno del modello(`Model year` in `canada_emissions` e `year` in `uk_emissions`), una feature che può essere importante per la predezioni delle emissioni prodotte. Quindi a partire dagli altri due dataset, viene estratta la colonna dell'anno.

Prima di procedere all'inserimento è necessario un adattamento della colonna `model`, poichè nel dataset `spain_emissions` ciascun valore è formato dal nome della casa automobilistica, ovvero il valore della colonna `make`, dal nome effettivo del modello e da altre informazioni aggiuntive sul modello. Quindi, si procede all'eliminazione della casa automobilistica e delle informazioni aggiuntive per adattarlo alla colonna del modello degli altri due dataset.

La prossime due celle rimuovono suffissi non presenti negli altri due dataset, eliminando le parole contenute nella variabile locale `useless_words`.

In [ ]:
useless_words = {"Canarias", "Vehículos", "Turismos", "Comerciales", "Nuevo", "NUEVO", "Turismos"}
# Per rimuovere suffissi fuorvianti, non presenti negli altri dataset, nei nomi delle case automobilistiche
def remove_useless_suffix(make):
    make_words = make.split()
    make_words = set(make_words) - useless_words
    return " ".join(make_words)

In [ ]:
spain_emissions["make"] = spain_emissions["make"].apply(remove_useless_suffix)

Di seguito, l'adattamento del nome del modello eliminando la casa automobilistica e le informazioni aggiuntive.

In [ ]:
# Per rimuovere il nome della casa automobilistica e le informazioni aggiuntive dal nome del modello
def remove_manufacturer_and_addional_info(make_model):
    make = make_model["make"]
    model = make_model["model"]
    removedManufacturer = model.replace(make, "").split()
    model = [word for word in removedManufacturer if word not in useless_words][0]
    return model

In [ ]:
spain_emissions["model"] = spain_emissions[["make", "model"]].apply(remove_manufacturer_and_addional_info, axis=1)

Dopo l'adattamento del nome della casa automobilistica e del nome del modello del veicolo, si può procedere all'effettivo inserimento dell'anno in base ai valori dell'anno nei dataset `uk_emissions` e `canada_emissions`. I valori dell'anno che risultano non presenti a seguito del join con `uk_emissions` verranno prelevati dal secondo dataset `canada_emissions`. In caso dei valori risultassero ancora mancanti, tali righe verranno scartate dal dataset, poichè prive di un'informazione rilevante ai fini dell'addestramento del modello.

In [ ]:
spain_emissions = (
    pd.merge(
        spain_emissions,
        uk_emissions[["Manufacturer", "Model", "Year"]],
        how = "left",
        left_on = ["make", "model"],
        right_on =  ["Manufacturer", "Model"]
    )
    .drop_duplicates()
)[list(spain_emissions.columns) + ["Year"]]

In [ ]:
spain_emissions_year_na = spain_emissions[spain_emissions["Year"].isna()].drop("Year", axis=1)
spain_emissions_year_canada = (
    pd.merge(
        spain_emissions_year_na,
        canada_emissions[["Make", "Model", "Model year"]],
        how = "left",
        left_on = ["make", "model"],
        right_on =  ["Make", "Model"]
    )
    .drop_duplicates()
    .rename(columns = {"Model year": "Year"})
)[list(spain_emissions.columns)]

La cella a seguire inserisce i valori dell'anno ottenuti dal join con `canada_emissions` al posto dei valori mancanti a seguito del join con `uk_emissions`. Infine, le righe in cui la colonna `year` risulta ancora mancante dopo i due join verranno scartate.   

In [ ]:
spain_emissions["Year"] = spain_emissions["Year"].fillna(spain_emissions_year_canada["Year"])
spain_emissions = spain_emissions.dropna()

### Grafici di `spain_emissions`
Questa sezione contiene alcuni grafici che evidenziano alcuni aspetti del dataframe risultante `spain_emissions`.value_countsunique

L'istogramma seguente mostra la distribuzione delle emissioni di CO<sub>2</sub> nel dataset spagnolo. I valori si concentrano principalmente nell'intervallo [100, 200].

In [ ]:
spain_emissions["emissions_gCO2_km"].plot.hist(bins=20,title="CO\u2082 emissions distribution");

Il seguente grafico mostra la correlazione tra il consumo di carburante e le emissioni di CO<sub>2</sub>. Si evince che esiste una dipendenza tra i due valori, fatta eccezione per alcuni punti.

In [ ]:
spain_emissions.plot.scatter("consumption_l_100km", "emissions_gCO2_km", title="Fuel consumption and CO\u2082 emissions correlation");

Nel grafico a torta è rappresentata la distribuzione delle categorie di veicoli. Una metà è occupata da sole 4 categorie, mentre le altre 8 si spartiscono il restante 50%.

In [ ]:
spain_emissions["market_segment"].value_counts()[:-1].plot.pie(autopct="%.2f%%", title="Market segments distribution", shadow=True, label="", figsize=(10,10));

Nel seguente grafico è mostrata la relazione tra ciascuna categoria di veicoli e le emissioni di CO<sub>2</sub> corrispondenti. Sono presenti molti outlier sia nella parte inferiore che nella parte superiore di ciascun boxplot. Si può notare come i le medie dei valori di veicoli più pesanti e più inquinanti come "Lorry", "Van: Cargo" e "Luxury" siano molto più alti delle medie dei valori corrispondenti ai veicoli più leggeri e meno inquinanti come "Compact", "Minicompact" e "Mid-size". Questa osservazione è vera solo per le medie e i mediani, siccome i valori di ciascun boxplot coprono una buona parte dell'asse y, quindi alcuni veicoli "Compact" hanno valori più alti di alcuni "Lorry".

In [ ]:
spain_emissions.boxplot(column="emissions_gCO2_km", by="market_segment", showmeans=True, figsize=(18, 6)).set_title("CO\u2082 emissions grouped by market segment");

Mentre nel grafico successivo vengono mostrati gli stessi valori, ma in relazione alla tipologia di motore. Quasi tutti boxplot hanno i valori di media e mediana che si assestano tra 150 e 200, presentando però anche in questo caso molti outlier.

In [ ]:
spain_emissions.boxplot(column="emissions_gCO2_km", by="engine_type", showmeans=True, figsize=(18, 6)).set_title("CO\u2082 emissions grouped by market engine type");

### Scelta delle colonne di `spain_emissions` per l'addestramento
Questa ultima parte dell'omogeneizzazione di `spain_emissions` seleziona solo le colonne utilizzate nell'addestramento rinominandole e facendo le ultime modifiche ai valori delle colonne, in modo da utilizzare uno standard comune agli altri dataset.

In [ ]:
old_columns = ["Year", "make", "model", "engine_displacement_cm3", "consumption_l_100km", "engine_type", "emissions_gCO2_km", "transmission"]
new_columns = ["Year", "Manufacturer", "Model", "Engine_cm3", "Fuel_consumption", "Fuel_type", "CO2_Emissions", "Transmission_type"]
columns_dict = dict(zip(old_columns, new_columns))

In [ ]:
spain_emissions = spain_emissions[old_columns].rename(columns = columns_dict)

Da questo punto in poi, Liquefied Petroleum Gas (LPG) verrà semplicemente sostituito con LPG.

In [ ]:
spain_emissions["Fuel_type"] = spain_emissions["Fuel_type"].replace("Liquefied petroleum gas(LPG)", "LPG")

# **Omogeneizzazione di `canada_emissions`**

### Gestione tipi di carburante

All'interno del dataset proveniente dal ministero canadese sono presenti dati riguardanti il tipo di carburante (colonna "Fuel type").

La cella seguente mostra il numero di occorrenze di ogni lettera nella colonna Fuel type.

In [ ]:
canada_emissions["Fuel type"].value_counts()

Riportiamo sotto il significato delle lettere.

- `X`: Benzina normale (ha un indice di ottano più basso ed è quindi meno resistente alla detonazione)
- `Z`: Benzina premium (ha un indice di ottano più alto e offre una maggiore resistenza alla detonazione. È ideale per motori ad alte prestazioni, che hanno un rapporto di compressione                           più elevato e richiedono un carburante più stabile.)
- `D`: Diesel
- `E`: Etanolo(E85)
- `N`: gas naturale

Per motivi di compatibilità dei dataset è superfluo mantenere la distinzione tra benzina normale e benzina premium. Compattiamo quindi le due lettere X e Z in una parola unica: Petrol.
Inoltre trasformiamo anche E in Etanolo, N in Natural gas e D in Diesel per gli stessi motivi di confrontabilità.
Prima di iniziare a modificare il dataframe per poter effettuare tutte le doverose modifiche per renderlo confrontabile con gli altri dataset ne salviamo una copia per evitare di perdere quello originale che potrebbe tornare utile.

In [ ]:
canada_emissions_copy = canada_emissions.copy()
canada_emissions["Fuel type"] = canada_emissions["Fuel type"].replace({'X': 'Petrol', 'Z': 'Petrol', 'D': 'Diesel', 'N': 'Natural Gas', 'E': 'Ethanol'})
canada_emissions["Fuel type"].value_counts()

Il seguente grafico a torta mostra la percentuale dei diversi tipi di carburante presi in considerazione. Si può notare che la maggior parte dei dati riguardano veicoli a benzina.

In [ ]:
fuel_type_colors = {"Petrol": "yellow", "Ethanol": "red", "Diesel": "green", "Natural Gas": "blue"}
canada_fuel_type = canada_emissions["Fuel type"].value_counts()
canada_fuel_type.plot.pie(colors = canada_fuel_type.index.map(fuel_type_colors), shadow=True, figsize=(8,8))
plt.show()

### Eliminazione colonne superflue e rename delle feature importanti

E' inoltre doveroso eliminare le feature City, Highway, Combined che indicano rispettivamente il consumo di carburante: nelle strade di città, nelle autostrade e in entrambe. Eliminiamo inoltre la colonna superflua dell'id, quella del tipo veicolo (Vehicle class) e quella del numero di cilindri (Cylinders) del veicolo, non presente negli altri dataframe.

In [ ]:
canada_emissions = canada_emissions.drop(["_id","Highway (L/100 km)","City (L/100 km)", "Combined (L/100 km)", "Vehicle class", "Cylinders"], axis=1)
canada_emissions.head(5)

La celle seguente serve solo per modificare i nomi delle feature in modo che siano compatibili con gli altri dataset.

In [ ]:
canada_emissions = canada_emissions.rename(columns={"Model year": "Year","Make": "Manufacturer","Fuel type": "Fuel_type", "Transmission": "Transmission_type","Combined (mpg)": "Fuel_consumption", "CO2 emissions (g/km)": "CO2_Emissions"})
canada_emissions.head(5)

### Gestione del tipo di trasmissione

Le prossime celle servono per rendere confrontabili con gli altri dataframe i valori riguardanti il tipo di trasmissione. In particolare trasformiamo tutti i valori di trasmissioni inizianti per A in Automatic e tutti quelli inizianti per M in Manual eccetto per i valori AM (che sarebbero automated manual) che verranno eliminati per evitare incomprensioni dato che gli altri dataset distinguono solamente tra cambio manuale e automatico.

In [ ]:
index_to_drop = canada_emissions[canada_emissions["Transmission_type"].str.startswith("AM")].index
canada_emissions = canada_emissions.drop(index_to_drop)
canada_emissions["Transmission_type"] = canada_emissions["Transmission_type"].replace({r'^M.*': 'Manual',r'^A.*': 'Automatic' }, regex=True)
canada_emissions["Transmission_type"].value_counts()

Mostriamo ora i risultati appena ottenuti in un grafico a torta.

In [ ]:
canada_emissions["Transmission_type"].value_counts().plot.pie(autopct="%.2f%%", shadow=True)
plt.show()

Come ci aspettavamo grazie allì'uso della funzione `value_counts()` i veicoli a cambio automatico sono la maggioranza.

### Dataset risultante

Adattamento finale del nome del modello eliminando la casa automobilistica e le informazioni aggiuntive.

In [ ]:
# Per rimuovere il nome della casa automobilistica e le informazioni aggiuntive dal nome del modello
def remove_manufacturer(make_model):
    make = make_model["Manufacturer"]
    model = make_model["Model"]
    removedManufacturer = model.replace(make, "").split()
    model = [word for word in removedManufacturer if word not in useless_words][0]
    return model

In [ ]:
canada_emissions["Model"] = canada_emissions[["Manufacturer", "Model"]].apply(remove_manufacturer, axis=1)

Mostriamo di seguito il dataframe risultante `canada_emissions`

In [ ]:
canada_emissions.head(10)

# Unione dei dataset

In [ ]:
emissions = pd.concat([spain_emissions, canada_emissions, uk_emissions], ignore_index=True)

Infine, per i contenuti di tipo stringa come `Manufacturer` e `Model`, sarà necessaria una standardizzazione tra i diversi dataset. Lo standard scelto è la maiuscola per il primo carattere e la minuscola per gli altri caratteri.

In [ ]:
capitalize_columns = ["Manufacturer", "Model"]
emissions[capitalize_columns] = emissions[capitalize_columns].apply(lambda col: col.str.capitalize())

Nella prossima cella, mostriamo una breve descrizione in formato tabellare delle feature numeriche del dataset completo

In [ ]:
emissions.describe()

In [ ]:
emissions.info()

# Addestramento modelli
A seguito dell'esplorazione e dell'omogeneizzazione dei tre dataset, si può procedere all'addestramento dei modelli. I modelli verranno addestrati sulle seguenti feature indipendenti:
- `Year`: anno in cui è stato prodotto il veicolo
- `Manufacturer`: casa automobilistica che ha prodotto il veicolo
- `Model`: nome del modello del veicolo
- `Engine_cm3`: volume del motore in cm<sup>3</sup>
- `Transmission_type`: tipologia di trasmissione del veicolo, ovvero se il cambio della marcia è automatico o manuale
- `Fuel_type`: tipologia di carburante usato nel veicolo. Per esempio può essere benzina, diesel, benzina e ibrida ecc.
- `Fuel_consumption`: i litri di carburante consumati dal veicolo ogni 100 km

La variabile dipendente target dell'addestramento è `CO2_Emissions`, la quantità di CO<sub>2</sub> emessa dal veicolo in grammi per km.

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------- Data Preparation ----------
X = emissions.drop("CO2_Emissions", axis=1)
y = emissions["CO2_Emissions"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Create train pool and split into train (22500) + val (2500)
train_pool_n = 22500 + 2500
X_train_pool = X_train.sample(n=train_pool_n, random_state=42)
y_train_pool = y_train.loc[X_train_pool.index]

# final train and val sets (keep names X_train / y_train for compatibility)
X_train = X_train_pool.sample(n=22500, random_state=42)
y_train = y_train_pool.loc[X_train.index]

X_val = X_train_pool.drop(X_train.index)
y_val = y_train_pool.loc[X_val.index]

# Limit the number of samples in test
X_test = X_test.sample(n=2500, random_state=42)
y_test = y_test.loc[X_test.index]

categorical_features = ['Manufacturer', 'Model', 'Fuel_type', 'Transmission_type']
numerical_features = ['Year', 'Engine_cm3', 'Fuel_consumption']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(X_train[categorical_features])

def encode_df(df):
    enc = encoder.transform(df[categorical_features])
    enc_df = pd.DataFrame(
        enc, columns=encoder.get_feature_names_out(categorical_features), index=df.index
    )
    return pd.concat([df[numerical_features].reset_index(drop=True),
                      enc_df.reset_index(drop=True)], axis=1)

X_train_enc = encode_df(X_train)
X_val_enc = encode_df(X_val)
X_test_enc = encode_df(X_test)

def to_tensor(x_df, y_series):
    X_t = torch.tensor(x_df.values, dtype=torch.float32)
    y_t = torch.tensor(y_series.values, dtype=torch.float32).unsqueeze(1)
    return X_t, y_t

X_train_tensor, y_train_tensor = to_tensor(X_train_enc, y_train)
X_val_tensor, y_val_tensor = to_tensor(X_val_enc, y_val)
X_test_tensor, y_test_tensor = to_tensor(X_test_enc, y_test)

# Full training dataset
full_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

## Definizione dei Modelli: MLP e KAN

In questo blocco vengono definite due architetture di modelli per regressione:

* **`MLP` (Multi-Layer Perceptron)**: una rete neurale feed-forward costruita in modo flessibile a partire da:

  * `input_dim`: numero di feature in input;
  * `hidden_sizes`: lista con il numero di neuroni per ciascun layer nascosto;
  * `dropout`: tasso di dropout per regolarizzare l'addestramento.
    Ogni layer nascosto è seguito da un'attivazione `ReLU` e un livello di `Dropout`. L'output è uno scalare, indicato per problemi di regressione.

* **`build_kan`**: funzione che restituisce un modello **KAN (Kolmogorov–Arnold Networks)**, una rete neurale basata su un'architettura alternativa ai classici MLP, caratterizzata da:

  * una struttura definita tramite la lista `width`,
  * griglie di punti (`grid`) e grado del polinomio (`k`) per le interpolazioni,
  * seme casuale (`seed`) per la riproducibilità e
  * assegnazione del modello al corretto `device` (CPU o GPU).


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_sizes, dropout):
        super().__init__()
        layers = []
        dim = input_dim
        for hs in hidden_sizes:
            layers.append(nn.Linear(dim, hs))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            dim = hs
        layers.append(nn.Linear(dim, 1))
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

def build_kan(input_dim, width, grid, k, seed=0):
    model = KAN(
        width=[input_dim] + list(width) + [1],
        grid=grid,
        k=k,
        seed=seed,
        device=device
    )
    model.speed()  # enable efficient mode: disable symbolic branch
    return model

def build_random_forest(**params):
    return RandomForestRegressor(**params)

def build_xgboost(**params):
    return xgb.XGBRegressor(**params)

## Nested Random Search con Early Stopping

Questo blocco di codice implementa una procedura completa di **Nested Random Search** per la selezione di iperparametri di modelli di regressione in PyTorch, integrando una strategia di **Early Stopping** per migliorare l'efficienza del training.

* La classe `EarlyStopper` consente di interrompere l'addestramento anticipatamente se la loss di validazione non migliora per un numero di epoche definito (`patience`), riducendo il rischio di overfitting e velocizzando l'ottimizzazione.
* Le funzioni `train_epoch` ed `eval_loss` gestiscono rispettivamente il training e la valutazione della loss media su un dataset.
* La funzione principale `nested_random_search` esegue una **Nested Cross-Validation**, dove:

  * Il ciclo esterno (outer loop) valuta le prestazioni generali del modello su diversi split train/test.
  * Il ciclo interno (inner loop) esplora combinazioni casuali di iperparametri tramite `ParameterSampler` per ottimizzare la loss di validazione, utilizzando K-Fold CV.
* Per ogni fold esterno, viene selezionata la migliore combinazione di iperparametri trovata all’interno, con successivo riaddestramento sul training set esteso e valutazione finale sul test set.

Il risultato è una lista di tuple contenenti i migliori parametri di modello, parametri di training e loss finale per ciascun fold esterno, utile per valutare la **robustezza e generalizzazione** del modello selezionato.

In [ ]:
class EarlyStopper:
    def __init__(self, patience=3, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')

    def early_stop(self, val_loss):
        # Se la loss migliora (di almeno min_delta), resettiamo il counter
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            # Se la loss non migliora da 'patience' epoche, dobbiamo fermarci
            if self.counter >= self.patience:
                return True
        return False

def train_epoch(model, loader, optimizer, criterion, l2_lambda=0.0):
    model.train()
    total_loss = 0.0
    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)
        optimizer.zero_grad()
        loss = criterion(model(Xb), yb)

        if l2_lambda > 0:
            l2_reg = torch.tensor(0.).to(device)
            for param in model.parameters():
                l2_reg += torch.norm(param, 2)
            loss += l2_lambda * l2_reg

        loss.backward()
        optimizer.step()
        total_loss += loss.item() * Xb.size(0)
    return total_loss / len(loader.dataset)

def eval_loss(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for Xb, yb in loader:
            Xb, yb = Xb.to(device), yb.to(device)
            total_loss += criterion(model(Xb), yb).item() * Xb.size(0)
    return total_loss / len(loader.dataset)

def nested_random_search_neural(model_builder, param_dist, dataset,
                               outer_folds=5, inner_folds=3, n_iter=10,
                               early_patience=10, early_min_delta=1e-4):
    """
    Esegue Nested Random Search con Randomized Search interno per reti neurali (MLP e KAN)
    Args:
        model_builder: funzione che crea un modello dato model_params
        param_dist: dizionario di liste per ogni iperparametro (model_params + training_params)
        dataset: TensorDataset per il training
        outer_folds, inner_folds: numeri di fold per CV
        n_iter: numero di campioni di Random Search
        early_patience: numero di epoche di pazienza per Early Stopping
        early_min_delta: miglioramento minimo per considerare un progresso
    Returns:
        lista di tuple (best_model_params, best_training_params, test_loss) per ogni outer fold
    """
    # Chiavi riservate per training
    train_keys = ['lr', 'batch_size', 'l2_lambda']
    outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=42)
    results = []

    for train_idx, test_idx in outer_cv.split(range(len(dataset))):
        inner_train = Subset(dataset, train_idx)
        inner_test = Subset(dataset, test_idx)
        best_val_loss = float('inf')
        best_model_params, best_train_params = None, None

        # Random Search interno
        for params in ParameterSampler(param_dist, n_iter=n_iter, random_state=42):
            # Separa i parametri del modello da quelli di training
            model_params = {k: v for k, v in params.items() if k not in train_keys}
            train_params = {k: v for k, v in params.items() if k in train_keys}

            # Valutazione su inner_folds con Early Stopping
            inner_cv = KFold(n_splits=inner_folds, shuffle=True, random_state=42)
            val_losses = []
            for subtrain_idx, val_idx in inner_cv.split(range(len(inner_train))):
                subtrain = Subset(inner_train, subtrain_idx)
                valset = Subset(inner_train, val_idx)
                train_loader = DataLoader(subtrain, batch_size=train_params['batch_size'], shuffle=True)
                val_loader = DataLoader(valset, batch_size=train_params['batch_size'], shuffle=False)

                # Costruisci modello
                model = model_builder(**model_params)
                if hasattr(model, 'speed'):
                    model.speed()
                model.to(device)
                optimizer = optim.Adam(model.parameters(), lr=train_params['lr'], weight_decay=train_params.get('l2_lambda', 0.0))
                stopper = EarlyStopper(patience=early_patience, min_delta=early_min_delta)

                # Training
                for epoch in range(1000):
                    train_epoch(model, train_loader, optimizer, nn.MSELoss(), l2_lambda=train_params.get('l2_lambda', 0.0))
                    val_loss = eval_loss(model, val_loader, nn.MSELoss())
                    if stopper.early_stop(val_loss):
                        break

                # Loss di validazione
                val_losses.append(eval_loss(model, val_loader, nn.MSELoss()))

            mean_val = np.mean(val_losses)
            if mean_val < best_val_loss:
                best_val_loss = mean_val
                best_model_params = model_params
                best_train_params = train_params

        # Riaddestramento con i migliori parametri su tutto il sottoinsieme interno con Early Stopping
        full_train_loader = DataLoader(inner_train, batch_size=best_train_params['batch_size'], shuffle=True)
        test_loader = DataLoader(inner_test, batch_size=best_train_params['batch_size'], shuffle=False)

        final_model = model_builder(**best_model_params)
        if hasattr(final_model, 'speed'):
            final_model.speed()
        final_model.to(device)

        optimizer = optim.Adam(final_model.parameters(), lr=best_train_params['lr'], weight_decay=train_params.get('l2_lambda', 0.0))
        stopper = EarlyStopper(patience=early_patience, min_delta=early_min_delta)

        for epoch in range(1000):
            train_epoch(final_model, full_train_loader, optimizer, nn.MSELoss(), l2_lambda=best_train_params.get('l2_lambda', 0.0))
            if stopper.early_stop(eval_loss(final_model, full_train_loader, nn.MSELoss())):
                break

        test_loss = eval_loss(final_model, test_loader, nn.MSELoss())
        results.append((best_model_params, best_train_params, test_loss))

    return results

def nested_random_search_sklearn(model_builder, param_dist, X_data, y_data,
                                outer_folds=5, inner_folds=3, n_iter=10):
    """
    Nested Random Search per modelli sklearn e XGBoost
    """
    outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=42)
    results = []

    for train_idx, test_idx in outer_cv.split(X_data):
        X_inner_train, X_inner_test = X_data.iloc[train_idx], X_data.iloc[test_idx]
        y_inner_train, y_inner_test = y_data.iloc[train_idx], y_data.iloc[test_idx]

        best_val_mse = float('inf')
        best_params = None

        for params in ParameterSampler(param_dist, n_iter=n_iter, random_state=42):
            inner_cv = KFold(n_splits=inner_folds, shuffle=True, random_state=42)
            val_mses = []

            for subtrain_idx, val_idx in inner_cv.split(X_inner_train):
                X_subtrain, X_val = X_inner_train.iloc[subtrain_idx], X_inner_train.iloc[val_idx]
                y_subtrain, y_val = y_inner_train.iloc[subtrain_idx], y_inner_train.iloc[val_idx]

                model = model_builder(**params)
                model.fit(X_subtrain, y_subtrain)
                val_pred = model.predict(X_val)
                val_mse = mean_squared_error(y_val, val_pred)
                val_mses.append(val_mse)

            mean_val_mse = np.mean(val_mses)
            if mean_val_mse < best_val_mse:
                best_val_mse = mean_val_mse
                best_params = params

        # Riaddestramento con los mejores parámetros
        final_model = model_builder(**best_params)
        final_model.fit(X_inner_train, y_inner_train)
        test_pred = final_model.predict(X_inner_test)
        test_mse = mean_squared_error(y_inner_test, test_pred)

        results.append((best_params, {}, test_mse))

    return results

## Esecuzione del Nested Random Search su MLP e KAN

In questa sezione viene eseguita la **ricerca di iperparametri tramite Nested Random Search** per due diverse architetture di modelli:
**MLP (Multi-Layer Perceptron)** e **KAN (Kolmogorov–Arnold Networks)**.

### Definizione degli spazi degli iperparametri:

* `mlp_param_dist`: contiene combinazioni di dimensioni dei layer nascosti, tassi di dropout, learning rate, batch size e numero massimo di epoche per il training del modello MLP.
* `kan_param_dist`: definisce i parametri strutturali del modello KAN come larghezza dei layer (`width`), numero di punti griglia (`grid`), grado delle funzioni di base (`k`), e parametri di training.

### Esecuzione:

* Viene eseguita la funzione `nested_random_search` separatamente per ciascun modello.
* I risultati di ogni fold (configurazioni migliori e relativa test loss) vengono stampati a video per permettere un confronto diretto tra le due architetture.

Questo confronto sistematico consente di determinare **quale modello e configurazione di iperparametri offra le migliori prestazioni** su un problema di regressione, valutato con Cross-Validation stratificata e early stopping.


---

**Scelta del numero di iterazioni per RandomizedSearchCV con MLP grid**

Il grid ha:

- Configurazioni Totali:
$$
M = 72
$$

Supponiamo di voler avere una probabilità \( P = 0.90 \) di includere almeno una delle migliori \( k = 10 \) configurazioni tra queste 72.

Usiamo la formula:

$$
n = \frac{\ln(1 - P)}{\ln\left(1 - \frac{k}{M}\right)}
$$

Calcoliamo:

$$
n = \frac{\ln(1 - 0.90)}{\ln\left(1 - \frac{10}{72}\right)} = \frac{\ln(0.10)}{\ln\left(\frac{62}{72}\right)} = \frac{-2.3026}{\ln(\frac{62}{72})} \approx \frac{2.3026}{0.1495} \approx 15.40
$$

Quindi, con **15 iterazioni** di Randomized Search, si ha circa il 90% di probabilità di testare almeno una delle 10 migliori configurazioni, risparmiando molto rispetto a un Grid Search completo con 72 combinazioni.

---

**Scelta del numero di iterazioni per RandomizedSearchCV con KAN grid**

Il grid ha:

- Configurazioni Totali:
$$
M = 32
$$

Supponiamo di voler avere una probabilità \( P = 0.90 \) di includere almeno una delle migliori \( k = 10 \) configurazioni tra queste 32.

Usiamo la formula:

$$
n = \frac{\ln(1 - P)}{\ln\left(1 - \frac{k}{M}\right)}
$$

Calcoliamo:

$$
n = \frac{\ln(1 - 0.90)}{\ln\left(1 - \frac{10}{32}\right)} = \frac{\ln(0.10)}{\ln\left(\frac{22}{32}\right)} = \frac{-2.3026}{\ln(\frac{22}{32})} \approx \frac{2.3026}{0.3747} \approx 6.15
$$

Quindi, con **6 iterazioni** di Randomized Search, si ha circa il 90% di probabilità di testare almeno una delle 10 migliori configurazioni, risparmiando molto rispetto a un Grid Search completo con 32 combinazioni.

---

**Scelta del numero di iterazioni per RandomizedSearchCV con RandomForest grid**

Il grid ha:

- Configurazioni Totali:
$$
M = 144
$$

Supponiamo di voler avere una probabilità \( P = 0.90 \) di includere almeno una delle migliori \( k = 10 \) configurazioni tra queste 144.

Usiamo la formula:

$$
n = \frac{\ln(1 - P)}{\ln\left(1 - \frac{k}{M}\right)}
$$

Calcoliamo:

$$
n = \frac{\ln(1 - 0.90)}{\ln\left(1 - \frac{10}{144}\right)} = \frac{\ln(0.10)}{\ln\left(\frac{134}{144}\right)} = \frac{-2.3026}{\ln(\frac{134}{144})} \approx \frac{2.3026}{0.0720} \approx 31.98
$$

Quindi, con **32 iterazioni** di Randomized Search, si ha circa il 90% di probabilità di testare almeno una delle 10 migliori configurazioni, risparmiando molto rispetto a un Grid Search completo con 144 combinazioni.

---

**Scelta del numero di iterazioni per RandomizedSearchCV con XGBoost grid**

Il grid ha:

- Configurazioni Totali:
$$
M = 2916
$$

Supponiamo di voler avere una probabilità \( P = 0.90 \) di includere almeno una delle migliori \( k = 10 \) configurazioni tra queste 2916.

Usiamo la formula:

$$
n = \frac{\ln(1 - P)}{\ln\left(1 - \frac{k}{M}\right)}
$$

Calcoliamo:

$$
n = \frac{\ln(1 - 0.90)}{\ln\left(1 - \frac{10}{2916}\right)} = \frac{\ln(0.10)}{\ln\left(\frac{2906}{2916}\right)} = \frac{-2.3026}{\ln(\frac{2906}{2916})} \approx \frac{2.3026}{0.0034} \approx 677.23
$$

Quindi, con **677 iterazioni** di Randomized Search, si ha circa il 90% di probabilità di testare almeno una delle 10 migliori configurazioni, risparmiando molto rispetto a un Grid Search completo con 2916 combinazioni.

---

**Da dove viene la formula per stimare il numero di iterazioni nel Randomized Search?**

Per stimare quante iterazioni (`n`) sono necessarie per avere una certa probabilità \(P\) di includere almeno una configurazione tra le \(k\) migliori (su \(M\) totali), usiamo la seguente logica probabilistica:

1. Probabilità di *non* pescare una top-\(k\) in un singolo tentativo.
Se ci sono \(M\) configurazioni totali e \(k\) di esse sono “quasi ottimali”, la probabilità di *non* sceglierne una buona è:
$$
1 - \frac{k}{M}
$$

2. Probabilità di non pescarne *nessuna* in \(n\) tentativi indipendenti
$$
\left(1 - \frac{k}{M} \right)^n
$$

3. Probabilità di pescare **almeno una** delle top-\(k\)
$$
P(\text{≥1 top-}k) = 1 - \left(1 - \frac{k}{M} \right)^n
$$

4. Ricavare \(n\) dalla formula
$$
1 - \left(1 - \frac{k}{M} \right)^n = P
\quad \Longrightarrow \quad
n = \frac{\ln(1 - P)}{\ln\left(1 - \frac{k}{M} \right)}
$$

5. Approssimazione per $$ k \ll M $$
Poiché $$ \ln(1 - x) \approx -x $$ per \(x\) piccolo:
$$
n \approx - \frac{\ln(1 - P)}{k/M}
$$

In [ ]:
input_dim = X_train_tensor.shape[1]
mlp_param_dist = {
    'input_dim': [input_dim],
    'hidden_sizes': [(32,32), (64,64), (128,)],
    'dropout': [0.1, 0.2, 0.5],
    'lr': [1e-3, 1e-4],
    'batch_size': [32],
    'l2_lambda': [0.0, 1e-5, 1e-4, 1e-3]
}

kan_param_dist = {
    'input_dim': [input_dim],
    'width': [(8,4), (16,8)],
    'grid': [5, 10],
    'k': [2, 4],
    'seed': [0],
    'lr': [1e-3],
    'batch_size': [32],
    'l2_lambda': [0.0, 1e-5, 1e-4, 1e-3]
}

rf_param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [10, 20, 30],
    'min_samples_split': [10, 20, 30],
    'min_samples_leaf': [5, 10],
    'max_features': ['sqrt', 'log2'],
    'random_state': [42]
}

xgb_param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'reg_alpha': [0.5, 1.0, 2.0],
    'reg_lambda': [2.0, 5.0, 10.0],
    'random_state': [42]
}

In [ ]:
# ---------- Run Nested Random Search ----------
print("=== Nested Random Search Results ===\n")

print("MLP Results:")
mlp_results = nested_random_search_neural(
    lambda **p: MLP(**{k: v for k, v in p.items() if k in ['input_dim', 'hidden_sizes', 'dropout']}),
    mlp_param_dist,
    full_dataset,
    n_iter=15)
for i, (model_p, train_p, loss) in enumerate(mlp_results):
    print(f"Fold {i+1} - Model: {model_p}, Train: {train_p}, Test Loss: {loss:.4f}")

print("\nKAN Results:")
kan_results = nested_random_search_neural(
    lambda **p: build_kan(**{k: v for k, v in p.items() if k in ['input_dim', 'width', 'grid', 'k', 'seed']}),
    kan_param_dist,
    full_dataset,
    n_iter=6)
for i, (model_p, train_p, loss) in enumerate(kan_results):
    print(f"Fold {i+1} - Model: {model_p}, Train: {train_p}, Test Loss: {loss:.4f}")

In [ ]:
print("\nRandom Forest Results:")
rf_results = nested_random_search_sklearn(build_random_forest, rf_param_dist, X_train_enc, y_train, n_iter=32)
for i, (model_p, _, loss) in enumerate(rf_results):
    print(f"Fold {i+1} - Params: {model_p}, Test MSE: {loss:.4f}")

print("\nXGBoost Results:")
xgb_results = nested_random_search_sklearn(build_xgboost, xgb_param_dist, X_train_enc, y_train, n_iter=677)
for i, (model_p, _, loss) in enumerate(xgb_results):
    print(f"Fold {i+1} - Params: {model_p}, Test MSE: {loss:.4f}")

## Valutazione Finale dei Migliori Modelli: MLP vs KAN

Questa sezione si occupa di confrontare i **migliori modelli trovati** durante la ricerca annidata (`nested_random_search`) per le due architetture:

* Viene definita la funzione `evaluate_model`, che calcola l'**MSE (Mean Squared Error)** di un modello su un `DataLoader`.
* Si identificano i **migliori modelli MLP e KAN**, selezionando quelli con la più bassa *test loss* tra i risultati di cross-validation.
* I modelli vengono **riaddestrati** usando gli stessi dati di test (dell'ultimo fold esterno) per analizzare:

  * **Train loss** ad ogni epoca;
  * **Validation loss** ad ogni epoca.

Questa fase è utile per:

* Verificare se il modello si **adatta bene** al test set senza overfitting;
* Raccogliere dati da **visualizzare graficamente** (es. curva di apprendimento) per analizzare il comportamento delle due architetture nel tempo.


In [ ]:
def evaluate_model_neural(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for Xb, yb in data_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            loss = criterion(model(Xb), yb)
            total_loss += loss.item() * Xb.size(0)
    return total_loss / len(data_loader.dataset)

def count_params(model):
    if isinstance(model, MLP):
        try:
            return sum(p.numel() for p in model.parameters() if p.requires_grad)
        except Exception:
            return 0

    elif isinstance(model, KAN):
        try:
            if not model.width or len(model.width) < 2:
                return 0
            else:
                sum_edge_terms = 0
                for i in range(len(model.width) - 1):
                    Nl = model.width[i]
                    Nl_plus_1 = model.width[i+1]
                    if isinstance(Nl, list): Nl = Nl[0]
                    if isinstance(Nl_plus_1, list): Nl_plus_1 = Nl_plus_1[0]
                    G = model.grid
                    k = model.k
                    sum_edge_terms += Nl * Nl_plus_1 * (G + k - 1)
                return sum_edge_terms
        except Exception as e:
            print(f"Error calculating KAN parameters: {e}")
            return 0

    elif isinstance(model, RandomForestRegressor):
        total_nodes = 0
        if hasattr(model, 'estimators_'):
            for tree in model.estimators_:
                if hasattr(tree, 'tree_'):
                    total_nodes += tree.tree_.node_count
            return total_nodes
        else:
            return 0

    elif isinstance(model, xgb.XGBRegressor):
        total_nodes = 0

        try:
            booster = model.get_booster()
            tree_dumps = booster.get_dump(dump_format='json')

            def count_nodes_in_json_tree(node):
                count = 1
                if 'children' in node:
                    for child in node['children']:
                        count += count_nodes_in_json_tree(child)
                return count

            for tree_dump_str in tree_dumps:
                tree_json = json.loads(tree_dump_str)
                total_nodes += count_nodes_in_json_tree(tree_json)

            return total_nodes
        except Exception as e:
            print(f"Error calculating exact XGBoost complexity: {e}")
            return 0
    else:
        return 0

def get_predictions_neural(model, data_loader):
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for Xb, yb in data_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            preds.append(model(Xb).cpu().numpy())
            true.append(yb.cpu().numpy())
    return np.vstack(true), np.vstack(preds)

def compute_confidence_interval(data, confidence=0.95):
    n = len(data)
    if n <= 1:
        return np.mean(data) if n > 0 else np.nan, np.nan, np.nan
    mean, se = np.mean(data), stats.sem(data)
    h = se * stats.t.ppf((1 + confidence) / 2., n-1)
    return mean, mean - h, mean + h


def compute_metrics_neural(model, data_loader):
    y_true, y_pred = get_predictions_neural(model, data_loader)
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()

    mse_scores = (y_true_flat - y_pred_flat)**2
    mae_scores = np.abs(y_true_flat - y_pred_flat)
    mape_scores = np.abs((y_true_flat - y_pred_flat) / y_true_flat) * 100
    mape_scores = mape_scores[np.isfinite(mape_scores) & (y_true_flat != 0)]

    mse, mse_ci_lower, mse_ci_upper = compute_confidence_interval(mse_scores)
    r2 = r2_score(y_true_flat, y_pred_flat)
    mae, mae_ci_lower, mae_ci_upper = compute_confidence_interval(mae_scores)
    mape, mape_ci_lower, mape_ci_upper = compute_confidence_interval(mape_scores)
    max_err = max_error(y_true_flat, y_pred_flat)
    n = len(y_true_flat)
    k = data_loader.dataset[0][0].shape[0]
    p = count_params(model)
    try:
        r2 = r2_score(y_true_flat, y_pred_flat)
        if n - k - 1 > 0:
            r2_adj = 1 - (1 - r2) * (n - 1) / (n - k - 1)
        else:
             r2_adj = r2
    except ZeroDivisionError:
        r2_adj = r2
    return (mse, mse_ci_lower, mse_ci_upper), r2, r2_adj, (mae, mae_ci_lower, mae_ci_upper), (mape, mape_ci_lower, mape_ci_upper), max_err, p

def compute_metrics_sklearn(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_test_np = y_test.values.flatten()
    y_pred_np = y_pred.flatten()

    min_len = min(len(y_test_np), len(y_pred_np))
    y_test_np = y_test_np[:min_len]
    y_pred_np = y_pred_np[:min_len]


    mse_scores = (y_test_np - y_pred_np)**2
    mae_scores = np.abs(y_test_np - y_pred_np)
    mape_scores = np.abs((y_test_np - y_pred_np) / y_test_np) * 100
    mape_scores = mape_scores[np.isfinite(mape_scores) & (y_test_np != 0)]


    mse, mse_ci_lower, mse_ci_upper = compute_confidence_interval(mse_scores)
    r2 = r2_score(y_test_np, y_pred_np)
    mae, mae_ci_lower, mae_ci_upper = compute_confidence_interval(mae_scores)
    mape, mape_ci_lower, mape_ci_upper = compute_confidence_interval(mape_scores)
    max_err = max_error(y_test_np, y_pred_np)
    n = len(y_test_np)
    k = X_test.shape[1]
    try:
         if n - k - 1 > 0:
            r2_adj = 1 - (1 - r2) * (n - 1) / (n - k - 1)
         else:
             r2_adj = r2
    except (ZeroDivisionError, ValueError):
        r2_adj = r2
    p = count_params(model)
    return (mse, mse_ci_lower, mse_ci_upper), r2, r2_adj, (mae, mae_ci_lower, mae_ci_upper), (mape, mape_ci_lower, mape_ci_upper), max_err, p

## Valutazione Finale e Confronto tra MLP e KAN

In questa sezione, vengono confrontati i modelli MLP (Multi-Layer Perceptron) e KAN (Kolmogorov–Arnold Networks) utilizzando metriche di regressione e visualizzazioni grafiche per analizzare le loro prestazioni.

### Metriche di Valutazione

Vengono calcolate le seguenti metriche sul test set:

* **MSE (Mean Squared Error)**: misura l'errore quadratico medio tra le predizioni del modello e i valori reali. Un valore più basso indica una migliore accuratezza del modello.

* **R² (Coefficiente di Determinazione)**: indica la proporzione della varianza nei dati dipendenti che è prevedibile dalle variabili indipendenti. Un valore più alto (fino a 1) suggerisce una migliore capacità predittiva.

* **R² Aggiustato**: modifica il R² standard penalizzando l'aggiunta di variabili indipendenti non significative, fornendo una misura più accurata della bontà del modello, specialmente quando si confrontano modelli con un numero diverso di predittori.&#x20;

* **Numero di Parametri Addestrabili**: indica la complessità del modello; modelli con un numero inferiore di parametri sono generalmente preferibili se le prestazioni sono comparabili, poiché tendono a generalizzare meglio e sono meno suscettibili all'overfitting.

### Visualizzazioni

1. **Curve di Loss per Epoca**: grafici che mostrano l'andamento della loss di training e di validazione per ciascun modello nel corso delle epoche, utili per identificare fenomeni di overfitting o underfitting.

2. **Confronto tramite Grafici a Barre**:

   * **Numero di Parametri**: confronta la complessità dei modelli.
   * **R² Aggiustato**: valuta la capacità predittiva tenendo conto della complessità del modello.
   * **MSE di Test**: misura l'accuratezza delle predizioni sui dati di test.([Cross Validated][1])

Queste analisi forniscono una panoramica completa delle prestazioni e della complessità dei modelli MLP e KAN, facilitando la selezione del modello più adatto per il problema in esame.


In [ ]:
# Selezione dei migliori modelli
best_mlp_idx = min(range(len(mlp_results)), key=lambda i: mlp_results[i][2])
best_mlp_params, best_mlp_train, _ = mlp_results[best_mlp_idx]

best_kan_idx = min(range(len(kan_results)), key=lambda i: kan_results[i][2])
best_kan_params, best_kan_train, _ = kan_results[best_kan_idx]

best_rf_idx = min(range(len(rf_results)), key=lambda i: rf_results[i][2])
best_rf_params, _, _ = rf_results[best_rf_idx]

best_xgb_idx = min(range(len(xgb_results)), key=lambda i: xgb_results[i][2])
best_xgb_params, _, _ = xgb_results[best_xgb_idx]

# Creazione dei modelli finali
mlp_model = MLP(**best_mlp_params).to(device)
kan_model = build_kan(**best_kan_params).to(device)
rf_model = build_random_forest(**best_rf_params)
xgb_model = build_xgboost(**best_xgb_params)

# Addestramento dei modelli sklearn
rf_model.fit(X_train_enc.values, y_train.values)
xgb_model.fit(X_train_enc.values, y_train.values)

# Preparazione dei loaders per i modelli neurali
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
train_idx, _ = list(outer_cv.split(range(len(full_dataset))))[-1]
train_loader = DataLoader(
    Subset(full_dataset, train_idx),
    batch_size=32,
    shuffle=True
)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False
)

# Addestramento dei modelli neurali
criterion = nn.MSELoss()
stopper_mlp = EarlyStopper(patience=10, min_delta=1e-4)
stopper_kan = EarlyStopper(patience=10, min_delta=1e-4)

# Training MLP
optimizer_mlp = optim.Adam(mlp_model.parameters(), lr=best_mlp_train['lr'])
for epoch in range(1000):
    train_loss_mlp = train_epoch(mlp_model, train_loader, optimizer_mlp, criterion, l2_lambda=best_mlp_train.get('l2_lambda', 0.0))
    val_loss_mlp = eval_loss(mlp_model, test_loader, criterion)
    if stopper_mlp.early_stop(val_loss_mlp):
        print(f"MLP Early stopping at epoch {epoch}")
        break

# Training KAN
optimizer_kan = optim.Adam(kan_model.parameters(), lr=best_kan_train['lr'])
for epoch in range(1000):
    train_loss_kan = train_epoch(kan_model, train_loader, optimizer_kan, criterion, l2_lambda=best_kan_train.get('l2_lambda', 0.0))
    val_loss_kan = eval_loss(kan_model, test_loader, criterion)
    if stopper_kan.early_stop(val_loss_kan):
        print(f"KAN Early stopping at epoch {epoch}")
        break

# Calcolo delle metriche
(mse_mlp_test, mse_mlp_test_ci_lower, mse_mlp_test_ci_upper), r2_mlp_test, r2_adj_mlp_test, (mae_mlp_test, mae_mlp_test_ci_lower, mae_mlp_test_ci_upper), (mape_mlp_test, mape_mlp_test_ci_lower, mape_mlp_test_ci_upper), max_err_mlp_test, params_mlp = compute_metrics_neural(mlp_model, test_loader)
(mse_kan_test, mse_kan_test_ci_lower, mse_kan_test_ci_upper), r2_kan_test, r2_adj_kan_test, (mae_kan_test, mae_kan_test_ci_lower, mae_kan_test_ci_upper), (mape_kan_test, mape_kan_test_ci_lower, mape_kan_test_ci_upper), max_err_kan_test, params_kan = compute_metrics_neural(kan_model, test_loader)
(mse_rf_test, mse_rf_test_ci_lower, mse_rf_test_ci_upper), r2_rf_test, r2_adj_rf_test, (mae_rf_test, mae_rf_test_ci_lower, mae_rf_test_ci_upper), (mape_rf_test, mape_rf_test_ci_lower, mape_rf_test_ci_upper), max_err_rf_test, params_rf = compute_metrics_sklearn(rf_model, X_test_enc, y_test)
(mse_xgb_test, mse_xgb_test_ci_lower, mse_xgb_test_ci_upper), r2_xgb_test, r2_adj_xgb_test, (mae_xgb_test, mae_xgb_test_ci_lower, mae_xgb_test_ci_upper), (mape_xgb_test, mape_xgb_test_ci_lower, mape_xgb_test_ci_upper), max_err_xgb_test, params_xgb = compute_metrics_sklearn(xgb_model, X_test_enc, y_test)

(mse_mlp_train, mse_mlp_train_ci_lower, mse_mlp_train_ci_upper), r2_mlp_train, r2_adj_mlp_train, (mae_mlp_train, mae_mlp_train_ci_lower, mae_mlp_train_ci_upper), (mape_mlp_train, mape_mlp_train_ci_lower, mape_mlp_train_ci_upper), max_err_mlp_train, _ = compute_metrics_neural(mlp_model, train_loader)
(mse_kan_train, mse_kan_train_ci_lower, mse_kan_train_ci_upper), r2_kan_train, r2_adj_kan_train, (mae_kan_train, mae_kan_train_ci_lower, mae_kan_train_ci_upper), (mape_kan_train, mape_kan_train_ci_lower, mape_kan_train_ci_upper), max_err_kan_train, _ = compute_metrics_neural(kan_model, train_loader)
(mse_rf_train, mse_rf_train_ci_lower, mse_rf_train_ci_upper), r2_rf_train, r2_adj_rf_train, (mae_rf_train, mae_rf_train_ci_lower, mae_rf_train_ci_upper), (mape_rf_train, mape_rf_train_ci_lower, mape_rf_train_ci_upper), max_err_rf_train, _ = compute_metrics_sklearn(rf_model, X_train_enc, y_train)
(mse_xgb_train, mse_xgb_train_ci_lower, mse_xgb_train_ci_upper), r2_xgb_train, r2_adj_xgb_train, (mae_xgb_train, mae_xgb_train_ci_lower, mae_xgb_train_ci_upper), (mape_xgb_train, mape_xgb_train_ci_lower, mape_xgb_train_ci_upper), max_err_xgb_train, _ = compute_metrics_sklearn(xgb_model, X_train_enc, y_train)

# Crea Dataframe dei risultati
models = ['MLP', 'KAN', 'Random Forest', 'XGBoost']

results_df = pd.DataFrame({
    'Model': models,
    'MSE_Train': [mse_mlp_train, mse_kan_train, mse_rf_train, mse_xgb_train],
    'MSE_Train_CI_Lower': [mse_mlp_train_ci_lower, mse_kan_train_ci_lower, mse_rf_train_ci_lower, mse_xgb_train_ci_lower],
    'MSE_Train_CI_Upper': [mse_mlp_train_ci_upper, mse_kan_train_ci_upper, mse_rf_train_ci_upper, mse_xgb_train_ci_upper],
    'MSE_Test': [mse_mlp_test, mse_kan_test, mse_rf_test, mse_xgb_test],
    'MSE_Test_CI_Lower': [mse_mlp_test_ci_lower, mse_kan_test_ci_lower, mse_rf_test_ci_lower, mse_xgb_test_ci_lower],
    'MSE_Test_CI_Upper': [mse_mlp_test_ci_upper, mse_kan_test_ci_upper, mse_rf_test_ci_upper, mse_xgb_test_ci_upper],
    'R²_Train': [r2_mlp_train, r2_kan_train, r2_rf_train, r2_xgb_train],
    'R²_Test': [r2_mlp_test, r2_kan_test, r2_rf_test, r2_xgb_test],
    'R²_Adjusted_Train': [r2_adj_mlp_train, r2_adj_kan_train, r2_adj_rf_train, r2_adj_xgb_train],
    'R²_Adjusted_Test': [r2_adj_mlp_test, r2_adj_kan_test, r2_adj_rf_test, r2_adj_xgb_test],
    'MAE_Train': [mae_mlp_train, mae_kan_train, mae_rf_train, mae_xgb_train],
    'MAE_Train_CI_Lower': [mae_mlp_train_ci_lower, mae_kan_train_ci_lower, mae_rf_train_ci_lower, mae_xgb_train_ci_lower],
    'MAE_Train_CI_Upper': [mae_mlp_train_ci_upper, mae_kan_train_ci_upper, mae_rf_train_ci_upper, mae_xgb_train_ci_upper],
    'MAE_Test': [mae_mlp_test, mae_kan_test, mae_rf_test, mae_xgb_test],
    'MAE_Test_CI_Lower': [mae_mlp_test_ci_lower, mae_kan_test_ci_lower, mae_rf_test_ci_lower, mae_xgb_test_ci_lower],
    'MAE_Test_CI_Upper': [mae_mlp_test_ci_upper, mae_kan_test_ci_upper, mae_rf_test_ci_upper, mae_xgb_test_ci_upper],
    'MAPE_Train': [mape_mlp_train, mape_kan_train, mape_rf_train, mape_xgb_train],
    'MAPE_Train_CI_Lower': [mape_mlp_train_ci_lower, mape_kan_train_ci_lower, mape_rf_train_ci_lower, mape_xgb_train_ci_lower],
    'MAPE_Train_CI_Upper': [mape_mlp_train_ci_upper, mape_kan_train_ci_upper, mape_rf_train_ci_upper, mape_xgb_train_ci_upper],
    'MAPE_Test': [mape_mlp_test, mape_kan_test, mape_rf_test, mape_xgb_test],
    'MAPE_Test_CI_Lower': [mape_mlp_test_ci_lower, mape_kan_test_ci_lower, mape_rf_test_ci_lower, mape_xgb_test_ci_lower],
    'MAPE_Test_CI_Upper': [mape_mlp_test_ci_upper, mape_kan_test_ci_upper, mape_rf_test_ci_upper, mape_xgb_test_ci_upper],
    'Max_Error_Train': [max_err_mlp_train, max_err_kan_train, max_err_rf_train, max_err_xgb_train],
    'Max_Error_Test': [max_err_mlp_test, max_err_kan_test, max_err_rf_test, max_err_xgb_test],
    'Parameters': [params_mlp, params_kan, params_rf, params_xgb]
})

In [ ]:
def plot_regression_scores(scores):
    model_order = scores['Model'].tolist()
    palette = sns.color_palette("viridis", len(model_order))
    model_colors = {model: palette[i] for i, model in enumerate(model_order)}

    fig, axs = plt.subplots(2, 3, figsize=(20, 12))
    fig.tight_layout(pad=4.0)

    # --- Plot 1: R² Score (Train vs Test) ---
    axs[0, 0].set_title('R² Score (Train vs Test)')
    axs[0, 0].set_xlabel('R² Score')
    axs[0, 0].set_ylabel('Model')
    axs[0, 0].set_xlim(0, 1)

    bar_height = 0.4
    for i, model in enumerate(model_order):
        row = scores[scores['Model'] == model].iloc[0]
        y_pos = i - bar_height/2

        axs[0, 0].barh(
            y_pos,
            row['R²_Train'],
            height=bar_height,
            color=model_colors[model],
            label=f'{model} - Train' if i == 0 else ""
        )

        y_pos = i + bar_height/2
        axs[0, 0].barh(
            y_pos,
            row['R²_Test'],
            height=bar_height,
            color=model_colors[model],
            alpha=0.6,
            hatch='//',
            label=f'{model} - Test' if i == 0 else ""
        )

    axs[0, 0].legend(handles=[
        Patch(color='gray', label='Train'),
        Patch(color='gray', label='Test', hatch='//', alpha=0.6)
    ], title='Set', loc='lower right')

    axs[0, 0].set_yticks(range(len(model_order)))
    axs[0, 0].set_yticklabels(model_order)
    axs[0, 0].invert_yaxis()

    # --- Helper function for consistent bar plots with CIs ---
    def plot_barh_with_ci(ax, data, metric_col, ci_low_col, ci_high_col, title, model_order, model_colors, xlim_max=None):
        ax.set_title(title)
        data_ordered = data.set_index('Model').loc[model_order].reset_index()

        for i, row in data_ordered.iterrows():
            val = row.get(metric_col)
            if pd.isna(val):
                continue

            err_low = [val - row.get(ci_low_col, val)] if pd.notna(row.get(ci_low_col)) else [0]
            err_high = [row.get(ci_high_col, val) - val] if pd.notna(row.get(ci_high_col)) else [0]

            ax.barh(
                row['Model'],
                val,
                xerr=[err_low, err_high],
                capsize=5,
                color=model_colors[row['Model']]
            )

        ax.set_xlabel(title.split(' ')[0])
        ax.invert_yaxis()
        if xlim_max:
            ax.set_xlim(0, xlim_max)

    # --- Plot 2: MSE Test Score ± CI95% ---
    plot_barh_with_ci(axs[0, 1], scores, 'MSE_Test', 'MSE_Test_CI_Lower', 'MSE_Test_CI_Upper',
                      'MSE Test ± CI95%', model_order, model_colors)

    # --- Plot 3: MAE Test Score ± CI95% ---
    plot_barh_with_ci(axs[0, 2], scores, 'MAE_Test', 'MAE_Test_CI_Lower', 'MAE_Test_CI_Upper',
                      'MAE Test ± CI95%', model_order, model_colors)

    # --- Plot 4: MAPE Test Score ± CI95% ---
    plot_barh_with_ci(axs[1, 0], scores, 'MAPE_Test', 'MAPE_Test_CI_Lower', 'MAPE_Test_CI_Upper',
                      'MAPE (%) Test ± CI95%', model_order, model_colors)

    # --- Plot 5: R² Adjusted Test Score ---
    axs[1, 1].set_title('R² Adjusted Test')
    scores_ordered = scores.set_index('Model').loc[model_order].reset_index()

    bars = axs[1, 1].barh(scores_ordered['Model'], scores_ordered['R²_Adjusted_Test'],
                          color=[model_colors[m] for m in scores_ordered['Model']])
    axs[1, 1].set_xlabel('R² Adjusted')
    axs[1, 1].invert_yaxis()
    axs[1, 1].set_xlim(0, 1)

    # --- Plot 6: Model Complexity (Parameter Count) ---
    axs[1, 2].set_title('Model Complexity (Parameters)')

    bars = axs[1, 2].barh(scores_ordered['Model'], scores_ordered['Parameters'],
                          color=[model_colors[m] for m in scores_ordered['Model']])
    axs[1, 2].set_xlabel('Parameter Count')
    axs[1, 2].set_xscale('log')  # Log scale for parameter count
    axs[1, 2].invert_yaxis()

    for i, (bar, count) in enumerate(zip(bars, scores_ordered['Parameters'])):
        if pd.notna(count) and count > 0:
            axs[1, 2].text(bar.get_width() * 1.1,
                           bar.get_y() + bar.get_height()/2,
                           f'{int(count):,}',
                           va='center', fontsize=9)

    plt.show()

In [ ]:
print("\n=== TABELLA RIASSUNTIVA ===")
print(results_df.round(4))

print("\n=== PLOTTING SCORES ===")
plot_regression_scores(results_df)

In [ ]:
# 1) Define metrics and their optimization direction
metrics = {
    'MSE_Test': 'min',
    'R²_Test': 'max',
    'R²_Adjusted_Test': 'max',
    'MAE_Test': 'min',
    'MAPE_Test': 'min',
    'Max_Error_Test': 'min'
}

# 2) Build ranking DataFrame
df_ranks = results_df.set_index('Model')
ranks = pd.DataFrame(index=df_ranks.index)

# Calculate ranks for performance metrics
for metric, direction in metrics.items():
    if direction == 'max':
        # For 'max' metrics (higher is better), rank in descending order (rank 1 is best)
        ranks[f"{metric}_rank"] = df_ranks[metric].rank(ascending=False, method='average')
    elif direction == 'min':
        # For 'min' metrics (lower is better), rank in ascending order (rank 1 is best)
        ranks[f"{metric}_rank"] = df_ranks[metric].rank(ascending=True, method='average')

# Calculate complexity rank (lower parameter count is better)
ranks['Complexity_rank'] = df_ranks['Parameters'].rank(ascending=True, method='average')

# 3) Calculate weighted scores
# Performance score (average of performance ranks)
performance_cols = [col for col in ranks.columns if col.endswith('_rank') and col != 'Complexity_rank']
ranks['performance_score'] = ranks[performance_cols].mean(axis=1)

# Method 1: Equal weighting
ranks['equal_weight_score'] = ranks['performance_score'] + ranks['Complexity_rank']

# Method 2: Complexity heavily weighted (complexity counts 2x)
ranks['complexity_weighted_score'] = ranks['performance_score'] + (2 * ranks['Complexity_rank'])

# Method 3: Extreme complexity weighting (complexity counts 3x)
ranks['extreme_complexity_score'] = ranks['performance_score'] + (3 * ranks['Complexity_rank'])

# Method 4: Pareto efficiency approach (performance vs complexity)
# Normalize scores to [0,1] for fair comparison
performance_norm = (ranks['performance_score'] - ranks['performance_score'].min()) / (ranks['performance_score'].max() - ranks['performance_score'].min())
complexity_norm = (ranks['Complexity_rank'] - ranks['Complexity_rank'].min()) / (ranks['Complexity_rank'].max() - ranks['Complexity_rank'].min())
ranks['pareto_score'] = 0.4 * performance_norm + 0.6 * complexity_norm  # 60% weight on complexity

# Display results for each method
methods = {
    'Equal Weight (1:1)': 'equal_weight_score',
    'Complexity Weighted (1:2)': 'complexity_weighted_score',
    'Extreme Complexity (1:3)': 'extreme_complexity_score',
    'Pareto Approach (40:60)': 'pareto_score'
}

results_summary = pd.DataFrame(index=df_ranks.index)
results_summary['Performance_Score'] = ranks['performance_score']
results_summary['Complexity_Rank'] = ranks['Complexity_rank']
results_summary['Parameters'] = df_ranks['Parameters']

print("---")
print("## Best Models by Weighting Scheme")
print("---")
best_models_summary_data = {
    'Weighting Scheme': [],
    'Best Model(s)': []
}

for method_name, score_col in methods.items():
    # For all these scores, a lower value is better
    best_model = ranks[score_col].idxmin()
    best_models_summary_data['Weighting Scheme'].append(method_name)
    best_models_summary_data['Best Model(s)'].append(best_model)

summary_df = pd.DataFrame(best_models_summary_data)
print(summary_df.to_markdown(index=False))

print("\n---")
print("## Detailed Ranking Table")
print("---")

# Create comprehensive ranking table
ranking_display = pd.DataFrame(index=df_ranks.index)
ranking_display['Parameters'] = df_ranks['Parameters'].astype(int)
ranking_display['Avg_Performance_Rank'] = ranks['performance_score'].round(2)
ranking_display['Complexity_Rank'] = ranks['Complexity_rank'].astype(int)

for method_name, score_col in methods.items():
    ranking_display[f'{method_name.split()[0]}_Rank'] = ranks[score_col].rank().astype(int)

# Sort by complexity-weighted score (our recommended approach for balancing performance and complexity)
ranking_display_sorted = ranking_display.sort_values('Complexity_Rank')
print(ranking_display_sorted.to_markdown())

print("\n---")
print("## Recommendation")
print("---")

# Our recommended model (complexity weighted approach)
recommended_model = ranks['complexity_weighted_score'].idxmin()
recommended_score = ranks.loc[recommended_model, 'complexity_weighted_score']
recommended_params = df_ranks.loc[recommended_model, 'Parameters']
# Assuming MSE_Test is a key metric to show for regression models
recommended_mse = df_ranks.loc[recommended_model, 'MSE_Test']

print(f"**RECOMMENDED MODEL:** {recommended_model}")
print(f"**Reason:** This model offers the best balance between predictive performance and model complexity.")
print(f"**Parameters:** {int(recommended_params):,}")
print(f"**MSE Test Score:** {recommended_mse:.4f}")
print(f"**Complexity-Weighted Rank Score:** {recommended_score:.3f}")

print(f"\n**TOP 3 MODELS** (Based on Complexity-Weighted Ranking):")
top_3 = ranks.sort_values('complexity_weighted_score').head(3)
for i, (model, row) in enumerate(top_3.iterrows(), 1):
    params = int(df_ranks.loc[model, 'Parameters'])
    mse_score = df_ranks.loc[model, 'MSE_Test']
    print(f" {i}. **{model}** | Parameters: {params:>8,} | MSE: {mse_score:.4f} | Score: {row['complexity_weighted_score']:.3f}")

# Ablation Study MLP e KAN

In [ ]:
class RegressionPruningAblationStudy:
    def __init__(self, device='cpu'):
        self.device = device
        self.pruning_results = []

    def get_model_sparsity(self, model):
        """Calcola la sparsità del modello (solo componenti MLP/KAN)"""
        if isinstance(model, MLP):
            # Per MLP, calcola sparsità di tutti i layer Linear
            total_params = 0
            zero_params = 0

            for module in model.modules():
                if isinstance(module, torch.nn.Linear):
                    if hasattr(module, 'weight'):
                        total_params += module.weight.numel()
                        zero_params += float(torch.sum(module.weight == 0))
                    if hasattr(module, 'bias') and module.bias is not None:
                        total_params += module.bias.numel()
                        zero_params += float(torch.sum(module.bias == 0))

            return zero_params / total_params if total_params > 0 else 0.0

        elif hasattr(model, 'width'):  # KAN model
            try:
                if not model.width or len(model.width) < 2:
                    return 0.0

                # Calcola parametri totali KAN
                total_params = count_params(model)
                zero_params = 0

                # Conta i parametri zero nella componente KAN
                for i in range(len(model.width) - 1):
                    if i < len(model.act_fun):
                        layer = model.act_fun[i]
                        # Accedi ai coefficienti spline (coef parameter)
                        if hasattr(layer, 'coef') and layer.coef is not None:
                            zero_params += float(torch.sum(layer.coef == 0))

                return zero_params / total_params if total_params > 0 else 0.0

            except Exception as e:
                print(f"  Error calculating KAN sparsity: {e}")
                return 0.0
        else:
            return 0.0

    def count_active_parameters(self, model):
        """Conta i parametri attivi nel modello"""
        if isinstance(model, MLP):
            active_params = 0
            for module in model.modules():
                if isinstance(module, torch.nn.Linear):
                    if hasattr(module, 'weight'):
                        active_params += float(torch.sum(module.weight != 0))
                    if hasattr(module, 'bias') and module.bias is not None:
                        active_params += float(torch.sum(module.bias != 0))
            return int(active_params)

        elif hasattr(model, 'width'):  # KAN model
            try:
                active_params = 0
                for i in range(len(model.width) - 1):
                    if i < len(model.act_fun):
                        layer = model.act_fun[i]
                        if hasattr(layer, 'coef') and layer.coef is not None:
                            active_params += float(torch.sum(layer.coef != 0))
                return int(active_params)
            except:
                return count_params(model)
        else:
            return count_params(model)

    def apply_l1_pruning(self, model, pruning_ratio):
      if pruning_ratio == 0.0:
          return copy.deepcopy(model)

      pruned_model = copy.deepcopy(model)

      if isinstance(model, MLP):
          modules_to_prune = []
          for module in pruned_model.modules():
              if isinstance(module, torch.nn.Linear):
                  modules_to_prune.append((module, 'weight'))
                  if hasattr(module, 'bias') and module.bias is not None:
                      modules_to_prune.append((module, 'bias'))

          if modules_to_prune:
              prune.global_unstructured(
                  modules_to_prune,
                  pruning_method=prune.L1Unstructured,
                  amount=pruning_ratio,
              )
              for module, param_name in modules_to_prune:
                  prune.remove(module, param_name)

          print(f"  Applied L1 pruning to MLP with ratio: {pruning_ratio:.3f}")
          return pruned_model

      else:  # KAN
          try:
              # Collect all trainable parameters
              all_params = []
              param_refs = []

              for name, param in pruned_model.named_parameters():
                  if param.requires_grad:
                      all_params.append(param.data.flatten())
                      param_refs.append((name, param))

              if not all_params:
                  print(f"  No trainable parameters found")
                  return model

              # Concatenate all parameters
              all_params_tensor = torch.cat(all_params)

              # Only use non-zero parameters for threshold calculation
              nonzero_mask = torch.abs(all_params_tensor) > 1e-10  # Small epsilon for numerical stability
              nonzero_params = all_params_tensor[nonzero_mask]

              if len(nonzero_params) == 0:
                  print(f"  Model already completely sparse")
                  return pruned_model

              # Calculate threshold from non-zero parameters only
              abs_nonzero = torch.abs(nonzero_params)
              threshold = torch.quantile(abs_nonzero, pruning_ratio).item()

              if threshold == 0.0:
                  # Fallback: use smallest non-zero value as threshold
                  threshold = abs_nonzero.min().item() * 1.001  # Slightly above minimum

              print(f"  Threshold: {threshold:.8f}, Non-zero params: {len(nonzero_params)}/{len(all_params_tensor)}")

              # Apply pruning to each parameter
              total_pruned = 0
              original_nonzero = len(nonzero_params)

              for name, param in param_refs:
                  # Create mask: keep if |param| >= threshold
                  mask = torch.abs(param.data) >= threshold

                  # Count parameters that will be pruned
                  will_be_pruned = torch.sum((torch.abs(param.data) > 1e-10) & (~mask)).item()
                  total_pruned += will_be_pruned

                  # Apply mask
                  param.data = param.data * mask

              actual_ratio = total_pruned / original_nonzero if original_nonzero > 0 else 0
              print(f"  Applied L1 pruning to KAN: {total_pruned}/{original_nonzero} params pruned (ratio: {actual_ratio:.3f})")

              return pruned_model

          except Exception as e:
              print(f"  Error during KAN pruning: {e}")
              return model

    def evaluate_pruned_model(self, model, model_name, test_loader, train_loader):
        """Valuta le prestazioni di un modello pruned per regressione"""
        model.eval()

        def get_predictions(data_loader):
            preds, true = [], []
            with torch.no_grad():
                for Xb, yb in data_loader:
                    Xb, yb = Xb.to(self.device), yb.to(self.device)
                    outputs = model(Xb)
                    preds.append(outputs.cpu().numpy())
                    true.append(yb.cpu().numpy())
            return np.vstack(true).flatten(), np.vstack(preds).flatten()

        # Predizioni su test set
        y_true_test, y_pred_test = get_predictions(test_loader)

        # Predizioni su train set
        y_true_train, y_pred_train = get_predictions(train_loader)

        # Calcola metriche per test set
        mse_test = mean_squared_error(y_true_test, y_pred_test)
        mae_test = mean_absolute_error(y_true_test, y_pred_test)
        r2_test = r2_score(y_true_test, y_pred_test)
        max_err_test = max_error(y_true_test, y_pred_test)

        # MAPE test
        mape_test = np.mean(np.abs((y_true_test - y_pred_test) / y_true_test)) * 100
        mape_test = mape_test if np.isfinite(mape_test) else np.nan

        # Calcola metriche per train set
        mse_train = mean_squared_error(y_true_train, y_pred_train)
        mae_train = mean_absolute_error(y_true_train, y_pred_train)
        r2_train = r2_score(y_true_train, y_pred_train)
        max_err_train = max_error(y_true_train, y_pred_train)

        # MAPE train
        mape_train = np.mean(np.abs((y_true_train - y_pred_train) / y_true_train)) * 100
        mape_train = mape_train if np.isfinite(mape_train) else np.nan

        return {
            'model_name': model_name,
            'mse_test': mse_test,
            'mae_test': mae_test,
            'r2_test': r2_test,
            'max_error_test': max_err_test,
            'mape_test': mape_test,
            'mse_train': mse_train,
            'mae_train': mae_train,
            'r2_train': r2_train,
            'max_error_train': max_err_train,
            'mape_train': mape_train
        }

    def run_pruning_study(self, model, model_name, test_loader, train_loader,
                         pruning_ratios=[0.0, 0.1, 0.2, 0.3, 0.5, 0.7, 0.8, 0.9, 0.95]):
        """
        Conduce lo studio di ablazione con L1 pruning per modelli di regressione
        """
        print(f"\n=== L1 Pruning Study for {model_name} ===")

        # Parametri originali
        original_params = count_params(model)
        print(f"Original Parameters: {original_params:,}")

        for pruning_ratio in pruning_ratios:
            print(f"\nTesting pruning ratio: {pruning_ratio:.4f}")

            # Applica pruning
            pruned_model = self.apply_l1_pruning(model, pruning_ratio)

            # Calcola sparsità e parametri attivi
            sparsity = self.get_model_sparsity(pruned_model)
            active_params = self.count_active_parameters(pruned_model)

            # Valuta prestazioni
            metrics = self.evaluate_pruned_model(
                pruned_model, model_name, test_loader, train_loader
            )

            # Calcola compression ratio
            compression_ratio = original_params / active_params if active_params > 0 else float('inf')

            # Salva risultati
            result = {
                'model_name': model_name,
                'pruning_ratio': pruning_ratio,
                'sparsity': sparsity,
                'original_params': original_params,
                'active_params': active_params,
                'compression_ratio': compression_ratio,
                'mse_test': metrics['mse_test'],
                'mae_test': metrics['mae_test'],
                'r2_test': metrics['r2_test'],
                'max_error_test': metrics['max_error_test'],
                'mape_test': metrics['mape_test'],
                'mse_train': metrics['mse_train'],
                'mae_train': metrics['mae_train'],
                'r2_train': metrics['r2_train'],
                'max_error_train': metrics['max_error_train'],
                'mape_train': metrics['mape_train']
            }

            self.pruning_results.append(result)

            print(f"  Sparsity: {sparsity:.3f}")
            print(f"  Active params: {active_params:,} / {original_params:,}")
            print(f"  Compression: {compression_ratio:.2f}x")
            print(f"  Test MSE: {metrics['mse_test']:.4f}")
            print(f"  Test R²: {metrics['r2_test']:.4f}")
            print(f"  Test MAE: {metrics['mae_test']:.4f}")

    def plot_pruning_results(self, figsize=(20, 12)):
        """
        Visualizza i risultati dello studio di pruning per regressione
        """
        if not self.pruning_results:
            print("No pruning results to plot. Run pruning study first.")
            return

        df = pd.DataFrame(self.pruning_results)

        fig, axes = plt.subplots(3, 4, figsize=figsize)
        fig.suptitle('L1 Pruning Study', fontsize=16, fontweight='bold')

        models = df['model_name'].unique()
        colors = sns.color_palette("husl", len(models))

        # Plot 1: MSE vs Pruning Ratio
        ax = axes[0, 0]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            ax.plot(model_data['pruning_ratio'], model_data['mse_test'],
                   marker='o', label=f'{model} (Test)', color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('MSE')
        ax.set_title('MSE vs Pruning Ratio')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_yscale('log')

        # Plot 2: R² vs Pruning Ratio
        ax = axes[0, 1]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            ax.plot(model_data['pruning_ratio'], model_data['r2_test'],
                   marker='o', label=f'{model} (Test)', color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('R² Score')
        ax.set_title('R² vs Pruning Ratio')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Plot 3: MAE vs Pruning Ratio
        ax = axes[0, 2]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            ax.plot(model_data['pruning_ratio'], model_data['mae_test'],
                   marker='o', label=f'{model} (Test)', color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('MAE')
        ax.set_title('MAE vs Pruning Ratio')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Plot 4: MAPE vs Pruning Ratio
        ax = axes[0, 3]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            valid_mape_test = model_data[model_data['mape_test'].notna()]
            valid_mape_train = model_data[model_data['mape_train'].notna()]
            if len(valid_mape_test) > 0:
                ax.plot(valid_mape_test['pruning_ratio'], valid_mape_test['mape_test'],
                       marker='o', label=f'{model} (Test)', color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('MAPE (%)')
        ax.set_title('MAPE vs Pruning Ratio')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Plot 5: Performance vs Compression Ratio
        ax = axes[1, 0]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            finite_mask = np.isfinite(model_data['compression_ratio'])
            if finite_mask.any():
                ax.scatter(model_data.loc[finite_mask, 'compression_ratio'],
                          model_data.loc[finite_mask, 'r2_test'],
                          label=model, color=colors[i], s=50, alpha=0.7)
        ax.set_xlabel('Compression Ratio (x)')
        ax.set_ylabel('R² Test Score')
        ax.set_title('Performance vs Compression')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_xscale('log')

        # Plot 6: Sparsity vs Performance
        ax = axes[1, 1]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            ax.plot(model_data['sparsity'], model_data['r2_test'],
                   marker='d', label=model, color=colors[i], linewidth=2)
        ax.set_xlabel('Sparsity')
        ax.set_ylabel('R² Test Score')
        ax.set_title('Performance vs Sparsity')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Plot 7: Parameters Count vs Pruning
        ax = axes[1, 2]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            ax.plot(model_data['pruning_ratio'], model_data['active_params'],
                   marker='o', label=model, color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('Active Parameters')
        ax.set_title('Active Parameters vs Pruning')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_yscale('log')

        # Plot 8: Performance Retention
        ax = axes[1, 3]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model].sort_values('pruning_ratio')
            baseline_r2 = model_data.iloc[0]['r2_test']
            performance_retention = model_data['r2_test'] / baseline_r2
            ax.plot(model_data['pruning_ratio'], performance_retention,
                   marker='s', label=model, color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('R² Retention (Relative)')
        ax.set_title('Performance Retention vs Pruning')
        ax.axhline(y=0.95, color='red', linestyle='--', alpha=0.7, label='95% threshold')
        ax.axhline(y=0.90, color='orange', linestyle='--', alpha=0.7, label='90% threshold')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Plot 9: Max Error vs Pruning
        ax = axes[2, 0]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            ax.plot(model_data['pruning_ratio'], model_data['max_error_test'],
                   marker='o', label=f'{model} (Test)', color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('Max Error')
        ax.set_title('Max Error vs Pruning Ratio')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_yscale('log')

        # Plot 10: Compression Efficiency
        ax = axes[2, 1]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            ax.plot(model_data['pruning_ratio'], model_data['compression_ratio'],
                   marker='o', label=model, color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('Compression Ratio (x)')
        ax.set_title('Compression Efficiency')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_yscale('log')

        axes[2, 2].axis('off')
        axes[2, 3].axis('off')

        plt.tight_layout()
        plt.show()

    def generate_pruning_report(self):
        """
        Genera un report dettagliato dei risultati del pruning per regressione
        """
        if not self.pruning_results:
            print("No pruning results available. Run pruning study first.")
            return

        df = pd.DataFrame(self.pruning_results)

        print("\n" + "="*90)
        print("L1 PRUNING STUDY - DETAILED REPORT")
        print("="*90)

        for model_name in df['model_name'].unique():
            model_data = df[df['model_name'] == model_name].sort_values('pruning_ratio')

            print(f"\n{model_name} Results:")
            print("-" * 60)

            # Baseline metrics
            baseline_row = model_data.iloc[0]
            baseline_r2 = baseline_row['r2_test']
            baseline_mse = baseline_row['mse_test']
            original_params = baseline_row['original_params']

            print(f"Original Parameters: {original_params:,}")
            print(f"Baseline Test R²: {baseline_r2:.4f}")
            print(f"Baseline Test MSE: {baseline_mse:.4f}")

            # Find degradation point (>5% loss in R²)
            degradation_point = None
            for _, row in model_data.iterrows():
                r2_loss = (baseline_r2 - row['r2_test']) / abs(baseline_r2) if baseline_r2 != 0 else 0
                if r2_loss > 0.05:  # 5% degradation threshold
                    degradation_point = row['pruning_ratio']
                    break

            if degradation_point:
                print(f"Significant degradation starts at: {degradation_point:.1%} pruning")
            else:
                print("No significant degradation observed within tested range")

            # Best trade-off point (maximum compression with <2% R² loss)
            best_tradeoff = None
            for _, row in model_data.iterrows():
                r2_loss = (baseline_r2 - row['r2_test']) / abs(baseline_r2) if baseline_r2 != 0 else 0
                if r2_loss <= 0.02 and row['pruning_ratio'] > 0:
                    best_tradeoff = row

            if best_tradeoff is not None:
                print(f"\nBest trade-off point:")
                print(f"  Pruning ratio: {best_tradeoff['pruning_ratio']:.1%}")
                print(f"  Compression: {best_tradeoff['compression_ratio']:.1f}x")
                print(f"  Test R²: {best_tradeoff['r2_test']:.4f}")
                print(f"  Test MSE: {best_tradeoff['mse_test']:.4f}")
                print(f"  R² loss: {((baseline_r2 - best_tradeoff['r2_test'])/abs(baseline_r2)*100):.1f}%")

            # Maximum compression achieved
            max_compression = model_data['compression_ratio'].replace([np.inf, -np.inf], np.nan).max()
            if not np.isnan(max_compression):
                print(f"\nMaximum compression achieved: {max_compression:.1f}x")

        # Comparative summary table
        print(f"\n{'='*90}")
        print("COMPARATIVE SUMMARY TABLE - PRUNING EFFECTIVENESS")
        print("="*90)

        summary_rows = []
        for model_name in df['model_name'].unique():
            model_data = df[df['model_name'] == model_name]
            baseline = model_data[model_data['pruning_ratio'] == 0.0].iloc[0]

            # Find results at different pruning thresholds
            for target_ratio in [0.3, 0.5, 0.7, 0.9]:
                closest = model_data.iloc[(model_data['pruning_ratio'] - target_ratio).abs().argsort()].iloc[0]
                if abs(closest['pruning_ratio'] - target_ratio) < 0.1:  # If close enough
                    r2_loss = ((baseline['r2_test'] - closest['r2_test']) / abs(baseline['r2_test'])) * 100 if baseline['r2_test'] != 0 else 0
                    mse_increase = ((closest['mse_test'] - baseline['mse_test']) / baseline['mse_test']) * 100
                    summary_rows.append({
                        'Model': model_name,
                        'Pruning_Ratio': f"{target_ratio:.0%}",
                        'Compression': f"{closest['compression_ratio']:.1f}x",
                        'R²_Test': f"{closest['r2_test']:.4f}",
                        'MSE_Test': f"{closest['mse_test']:.2f}",
                        'R²_Loss': f"{r2_loss:.1f}%",
                        'MSE_Increase': f"{mse_increase:.1f}%"
                    })

        if summary_rows:
            summary_df = pd.DataFrame(summary_rows)
            print(summary_df.to_string(index=False))

print("Iniziando L1 Pruning Study per modelli di regressione...")
print("METODOLOGIA:")
print("- MLP: L1 norm pruning su tutti i layer Linear")
print("- KAN: L1 norm pruning sui coefficienti spline")
print("- Metriche: MSE, MAE, R², MAPE, Max Error\n")

# Inizializza la classe per lo studio di pruning
regression_pruning_study = RegressionPruningAblationStudy(device=device)

# Definisci i livelli di pruning da testare
pruning_ratios_uniform = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

# Esegui pruning study su MLP
if 'mlp_model' in locals():
    print("Trovato modello MLP - Eseguendo L1 pruning study...")
    regression_pruning_study.run_pruning_study(
        model=mlp_model,
        model_name='MLP',
        test_loader=test_loader,
        train_loader=train_loader,
        pruning_ratios=pruning_ratios_uniform
    )

# Esegui pruning study su KAN
if 'kan_model' in locals():
    print("Trovato modello KAN - Eseguendo L1 pruning study...")
    regression_pruning_study.run_pruning_study(
        model=kan_model,
        model_name='KAN',
        test_loader=test_loader,
        train_loader=train_loader,
        pruning_ratios=pruning_ratios_uniform
    )

# Visualizza i risultati
regression_pruning_study.plot_pruning_results()

# Genera report dettagliato
regression_pruning_study.generate_pruning_report()

# Salva i risultati in DataFrame per ulteriori analisi
pruning_results_df = pd.DataFrame(regression_pruning_study.pruning_results)
print(f"\nPruning results saved to 'pruning_results_df' with {len(pruning_results_df)} entries")
print("\nPruning results:")
display(pruning_results_df)

# Ablation Study Random Forest e XGBoost

In [ ]:
class EnsemblePruningAblationStudy:
    def __init__(self):
        self.ensemble_results = []

    def calculate_tree_importance_rf(self, model, X_val, y_val):
        """
        Calculate tree importance for Random Forest using validation performance
        """
        tree_importances = []

        for i, tree in enumerate(model.estimators_):
            # Make predictions with single tree
            tree_pred = tree.predict(X_val)

            # Calculate individual tree performance (negative MSE for ranking)
            tree_mse = mean_squared_error(y_val, tree_pred)
            tree_r2 = r2_score(y_val, tree_pred)

            # Use R² as importance metric (higher is better)
            tree_importances.append({
                'tree_idx': i,
                'importance': tree_r2,
                'mse': tree_mse,
                'r2': tree_r2
            })

        # Sort by importance (R² descending)
        tree_importances.sort(key=lambda x: x['importance'], reverse=True)
        return tree_importances

    def calculate_tree_importance_xgb(self, model, X_val, y_val):
        """
        Calculate cumulative importance for XGBoost trees using incremental R² improvement
        """
        booster = model.get_booster()
        n_estimators = model.n_estimators
        dval = xgb.DMatrix(X_val)
        tree_importances = []

        for i in range(n_estimators):
            # Predictions from first i+1 trees
            pred_i = booster.predict(dval, iteration_range=(0, i+1))
            r2_i = r2_score(y_val, pred_i)
            if i == 0:
                improvement = r2_i
            else:
                pred_prev = booster.predict(dval, iteration_range=(0, i))
                r2_prev = r2_score(y_val, pred_prev)
                improvement = r2_i - r2_prev

            tree_importances.append({
                'tree_idx': i,
                'importance': improvement,
                'cumulative_r2': r2_i,
                'cumulative_mse': mean_squared_error(y_val, pred_i)
            })
        return tree_importances

    def apply_rank_based_pruning_rf(self, model, pruning_ratio, X_val, y_val):
        """
        Apply rank-based pruning to Random Forest
        """
        if pruning_ratio == 0.0:
            return copy.deepcopy(model)

        # Calculate tree importances
        tree_importances = self.calculate_tree_importance_rf(model, X_val, y_val)

        # Determine how many trees to keep
        n_trees = len(model.estimators_)
        n_keep = max(1, int(n_trees * (1 - pruning_ratio)))

        # Select top trees
        top_trees_idx = [item['tree_idx'] for item in tree_importances[:n_keep]]

        # Create pruned model
        pruned_model = copy.deepcopy(model)
        pruned_model.estimators_ = [model.estimators_[i] for i in top_trees_idx]
        pruned_model.n_estimators = len(pruned_model.estimators_)

        return pruned_model

    def apply_cumulative_pruning_xgb(self, model, pruning_ratio, X_val, y_val):
        """
        Apply post-training cumulative pruning to XGBoost by limiting predictions to first n_keep trees
        """
        if pruning_ratio <= 0.0:
            return copy.deepcopy(model)

        # Calculate tree importances to find cutoff
        tree_importances = self.calculate_tree_importance_xgb(model, X_val, y_val)
        n_trees = len(tree_importances)
        n_keep = max(1, int(n_trees * (1 - pruning_ratio)))

        # Get original booster
        original_booster = model.get_booster()

        # Create pruned model wrapper
        pruned_model = copy.deepcopy(model)
        pruned_model.n_estimators = n_keep

        def predict_pruned(self, X):
            dmat = xgb.DMatrix(X)
            return original_booster.predict(dmat, iteration_range=(0, n_keep))
        pruned_model.predict = types.MethodType(predict_pruned, pruned_model)

        return pruned_model

    def count_ensemble_components(self, model):
        """Count the number of components in ensemble model"""
        if isinstance(model, RandomForestRegressor):
            return len(model.estimators_) if hasattr(model, 'estimators_') else model.n_estimators
        elif isinstance(model, xgb.XGBRegressor):
            return model.n_estimators
        else:
            return 0

    def calculate_ensemble_sparsity(self, original_count, pruned_count):
        """Calculate sparsity as fraction of components removed"""
        return (original_count - pruned_count) / original_count if original_count > 0 else 0.0

    def evaluate_ensemble_model(self, model, model_name, X_test, y_test, X_train, y_train):
        """Evaluate ensemble model performance"""

        # Test predictions
        y_pred_test = model.predict(X_test)
        y_test_np = y_test.values.flatten() if hasattr(y_test, 'values') else y_test.flatten()
        y_pred_test = y_pred_test.flatten()

        # Train predictions
        y_pred_train = model.predict(X_train)
        y_train_np = y_train.values.flatten() if hasattr(y_train, 'values') else y_train.flatten()
        y_pred_train = y_pred_train.flatten()

        # Ensure same length
        min_len_test = min(len(y_test_np), len(y_pred_test))
        y_test_np = y_test_np[:min_len_test]
        y_pred_test = y_pred_test[:min_len_test]

        min_len_train = min(len(y_train_np), len(y_pred_train))
        y_train_np = y_train_np[:min_len_train]
        y_pred_train = y_pred_train[:min_len_train]

        # Calculate test metrics
        mse_test = mean_squared_error(y_test_np, y_pred_test)
        mae_test = mean_absolute_error(y_test_np, y_pred_test)
        r2_test = r2_score(y_test_np, y_pred_test)
        max_err_test = max_error(y_test_np, y_pred_test)

        # MAPE test (handle division by zero)
        mape_test = np.mean(np.abs((y_test_np - y_pred_test) / y_test_np)) * 100
        mape_test = mape_test if np.isfinite(mape_test) else np.nan

        # Calculate train metrics
        mse_train = mean_squared_error(y_train_np, y_pred_train)
        mae_train = mean_absolute_error(y_train_np, y_pred_train)
        r2_train = r2_score(y_train_np, y_pred_train)
        max_err_train = max_error(y_train_np, y_pred_train)

        # MAPE train
        mape_train = np.mean(np.abs((y_train_np - y_pred_train) / y_train_np)) * 100
        mape_train = mape_train if np.isfinite(mape_train) else np.nan

        return {
            'model_name': model_name,
            'mse_test': mse_test,
            'mae_test': mae_test,
            'r2_test': r2_test,
            'max_error_test': max_err_test,
            'mape_test': mape_test,
            'mse_train': mse_train,
            'mae_train': mae_train,
            'r2_train': r2_train,
            'max_error_train': max_err_train,
            'mape_train': mape_train
        }

    def run_ensemble_pruning_study(self, model, model_name, X_train, y_train, X_test, y_test, X_val, y_val,
                                 pruning_ratios=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
        """
        Run ensemble pruning study
        """
        print(f"\n=== Ensemble Pruning Study for {model_name} ===")

        # Original component count
        original_components = self.count_ensemble_components(model)
        print(f"Original {model_name} components: {original_components}")

        for pruning_ratio in pruning_ratios:
            print(f"\nTesting pruning ratio: {pruning_ratio:.4f}")

            if isinstance(model, RandomForestRegressor):
                # Apply rank-based pruning
                pruned_model = self.apply_rank_based_pruning_rf(model, pruning_ratio, X_val, y_val)
            elif isinstance(model, xgb.XGBRegressor):
                # Apply cumulative pruning
                pruned_model = self.apply_cumulative_pruning_xgb(model, pruning_ratio, X_val, y_val)
            else:
                print(f"Unsupported model type: {type(model)}")
                continue

            # Count components after pruning
            pruned_components = self.count_ensemble_components(pruned_model)
            sparsity = self.calculate_ensemble_sparsity(original_components, pruned_components)
            compression_ratio = original_components / pruned_components if pruned_components > 0 else float('inf')

            # Evaluate performance
            metrics = self.evaluate_ensemble_model(
                pruned_model, model_name, X_test, y_test, X_train, y_train
            )

            # Store results
            result = {
                'model_name': model_name,
                'pruning_ratio': pruning_ratio,
                'sparsity': sparsity,
                'original_components': original_components,
                'active_components': pruned_components,
                'compression_ratio': compression_ratio,
                'mse_test': metrics['mse_test'],
                'mae_test': metrics['mae_test'],
                'r2_test': metrics['r2_test'],
                'max_error_test': metrics['max_error_test'],
                'mape_test': metrics['mape_test'],
                'mse_train': metrics['mse_train'],
                'mae_train': metrics['mae_train'],
                'r2_train': metrics['r2_train'],
                'max_error_train': metrics['max_error_train'],
                'mape_train': metrics['mape_train']
            }

            self.ensemble_results.append(result)

            print(f"  Active components: {pruned_components} / {original_components}")
            print(f"  Compression: {compression_ratio:.2f}x")
            print(f"  Test MSE: {metrics['mse_test']:.4f}")
            print(f"  Test R²: {metrics['r2_test']:.4f}")
            print(f"  Test MAE: {metrics['mae_test']:.4f}")

    def plot_ensemble_pruning_results(self, figsize=(20, 12)):
        """
        Plot ensemble pruning results
        """
        if not self.ensemble_results:
            print("No ensemble pruning results to plot. Run ensemble pruning study first.")
            return

        df = pd.DataFrame(self.ensemble_results)

        fig, axes = plt.subplots(3, 4, figsize=figsize)
        fig.suptitle('Ensemble Pruning Study - Random Forest (Rank-Based) & XGBoost (Cumulative)',
                    fontsize=16, fontweight='bold')

        models = df['model_name'].unique()
        colors = sns.color_palette("Set1", len(models))

        # Plot 1: MSE vs Pruning Ratio
        ax = axes[0, 0]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            ax.plot(model_data['pruning_ratio'], model_data['mse_test'],
                   marker='o', label=f'{model} (Test)', color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('MSE')
        ax.set_title('MSE vs Pruning Ratio')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_yscale('log')

        # Plot 2: R² vs Pruning Ratio
        ax = axes[0, 1]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            ax.plot(model_data['pruning_ratio'], model_data['r2_test'],
                   marker='o', label=f'{model} (Test)', color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('R² Score')
        ax.set_title('R² vs Pruning Ratio')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Plot 3: MAE vs Pruning Ratio
        ax = axes[0, 2]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            ax.plot(model_data['pruning_ratio'], model_data['mae_test'],
                   marker='o', label=f'{model} (Test)', color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('MAE')
        ax.set_title('MAE vs Pruning Ratio')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Plot 4: MAPE vs Pruning Ratio
        ax = axes[0, 3]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            valid_mape_test = model_data[model_data['mape_test'].notna()]
            if len(valid_mape_test) > 0:
                ax.plot(valid_mape_test['pruning_ratio'], valid_mape_test['mape_test'],
                       marker='o', label=f'{model} (Test)', color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('MAPE (%)')
        ax.set_title('MAPE vs Pruning Ratio')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Plot 5: Performance vs Compression Ratio
        ax = axes[1, 0]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            finite_mask = np.isfinite(model_data['compression_ratio'])
            if finite_mask.any():
                ax.scatter(model_data.loc[finite_mask, 'compression_ratio'],
                          model_data.loc[finite_mask, 'r2_test'],
                          label=model, color=colors[i], s=50, alpha=0.7)
        ax.set_xlabel('Compression Ratio (x)')
        ax.set_ylabel('R² Test Score')
        ax.set_title('Performance vs Compression')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_xscale('log')

        # Plot 6: Sparsity vs Performance
        ax = axes[1, 1]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            ax.plot(model_data['sparsity'], model_data['r2_test'],
                   marker='d', label=model, color=colors[i], linewidth=2)
        ax.set_xlabel('Sparsity (Fraction Removed)')
        ax.set_ylabel('R² Test Score')
        ax.set_title('Performance vs Sparsity')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Plot 7: Active Components vs Pruning
        ax = axes[1, 2]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            ax.plot(model_data['pruning_ratio'], model_data['active_components'],
                   marker='o', label=model, color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('Active Components (Trees/Estimators)')
        ax.set_title('Active Components vs Pruning')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_yscale('log')

        # Plot 8: Performance Retention
        ax = axes[1, 3]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model].sort_values('pruning_ratio')
            if len(model_data) > 0:
                baseline_r2 = model_data.iloc[0]['r2_test']
                if baseline_r2 != 0:
                    performance_retention = model_data['r2_test'] / baseline_r2
                    ax.plot(model_data['pruning_ratio'], performance_retention,
                           marker='s', label=model, color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('R² Retention (Relative)')
        ax.set_title('Performance Retention vs Pruning')
        ax.axhline(y=0.95, color='red', linestyle='--', alpha=0.7, label='95% threshold')
        ax.axhline(y=0.90, color='orange', linestyle='--', alpha=0.7, label='90% threshold')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Plot 9: Max Error vs Pruning
        ax = axes[2, 0]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            ax.plot(model_data['pruning_ratio'], model_data['max_error_test'],
                   marker='o', label=f'{model} (Test)', color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('Max Error')
        ax.set_title('Max Error vs Pruning Ratio')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_yscale('log')

        # Plot 10: Compression Efficiency
        ax = axes[2, 1]
        for i, model in enumerate(models):
            model_data = df[df['model_name'] == model]
            finite_mask = np.isfinite(model_data['compression_ratio'])
            if finite_mask.any():
                ax.plot(model_data.loc[finite_mask, 'pruning_ratio'],
                       model_data.loc[finite_mask, 'compression_ratio'],
                       marker='o', label=model, color=colors[i], linewidth=2)
        ax.set_xlabel('Pruning Ratio')
        ax.set_ylabel('Compression Ratio (x)')
        ax.set_title('Compression Efficiency')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_yscale('log')

        axes[2, 2].axis('off')
        axes[2, 3].axis('off')

        plt.tight_layout()
        plt.show()

    def generate_ensemble_pruning_report(self):
        """
        Generate detailed ensemble pruning report
        """
        if not self.ensemble_results:
            print("No ensemble pruning results available. Run ensemble pruning study first.")
            return

        df = pd.DataFrame(self.ensemble_results)

        print("\n" + "="*100)
        print("ENSEMBLE PRUNING STUDY - DETAILED REPORT")
        print("Random Forest: Rank-Based Pruning | XGBoost: Cumulative Pruning")
        print("="*100)

        for model_name in df['model_name'].unique():
            model_data = df[df['model_name'] == model_name].sort_values('pruning_ratio')

            print(f"\n{model_name} Results:")
            print("-" * 70)

            # Baseline metrics
            baseline_row = model_data.iloc[0]
            baseline_r2 = baseline_row['r2_test']
            baseline_mse = baseline_row['mse_test']
            original_components = baseline_row['original_components']

            print(f"Original Components: {original_components}")
            print(f"Baseline Test R²: {baseline_r2:.4f}")
            print(f"Baseline Test MSE: {baseline_mse:.4f}")

            # Pruning method description
            if model_name == 'Random Forest':
                print("Pruning Method: Rank-Based (keeping best performing trees)")
            elif model_name == 'XGBoost':
                print("Pruning Method: Cumulative (early stopping based approach)")

            # Find significant degradation point
            degradation_point = None
            for _, row in model_data.iterrows():
                r2_loss = (baseline_r2 - row['r2_test']) / abs(baseline_r2) if baseline_r2 != 0 else 0
                if r2_loss > 0.05:  # 5% degradation threshold
                    degradation_point = row['pruning_ratio']
                    break

            if degradation_point:
                print(f"Significant degradation starts at: {degradation_point:.1%} pruning")
            else:
                print("No significant degradation observed within tested range")

            # Best trade-off point
            best_tradeoff = None
            for _, row in model_data.iterrows():
                r2_loss = (baseline_r2 - row['r2_test']) / abs(baseline_r2) if baseline_r2 != 0 else 0
                if r2_loss <= 0.02 and row['pruning_ratio'] > 0:
                    best_tradeoff = row

            if best_tradeoff is not None:
                print(f"\nBest trade-off point:")
                print(f"  Pruning ratio: {best_tradeoff['pruning_ratio']:.1%}")
                print(f"  Compression: {best_tradeoff['compression_ratio']:.1f}x")
                print(f"  Active components: {best_tradeoff['active_components']}/{original_components}")
                print(f"  Test R²: {best_tradeoff['r2_test']:.4f}")
                print(f"  Test MSE: {best_tradeoff['mse_test']:.4f}")
                print(f"  R² loss: {((baseline_r2 - best_tradeoff['r2_test'])/abs(baseline_r2)*100):.1f}%")

            # Maximum compression achieved
            max_compression = model_data['compression_ratio'].replace([np.inf, -np.inf], np.nan).max()
            if not np.isnan(max_compression):
                print(f"\nMaximum compression achieved: {max_compression:.1f}x")

        # Comparative analysis
        print(f"\n{'='*100}")
        print("COMPARATIVE ANALYSIS - ENSEMBLE PRUNING METHODS")
        print("="*100)

        summary_rows = []
        for model_name in df['model_name'].unique():
            model_data = df[df['model_name'] == model_name]
            baseline = model_data[model_data['pruning_ratio'] == 0.0].iloc[0]

            for target_ratio in [0.3, 0.5, 0.7, 0.9]:
                closest = model_data.iloc[(model_data['pruning_ratio'] - target_ratio).abs().argsort()]
                if len(closest) > 0 and abs(closest.iloc[0]['pruning_ratio'] - target_ratio) < 0.1:
                    row = closest.iloc[0]
                    r2_loss = ((baseline['r2_test'] - row['r2_test']) / abs(baseline['r2_test'])) * 100 if baseline['r2_test'] != 0 else 0
                    mse_increase = ((row['mse_test'] - baseline['mse_test']) / baseline['mse_test']) * 100

                    summary_rows.append({
                        'Model': model_name,
                        'Pruning_Ratio': f"{target_ratio:.0%}",
                        'Components_Kept': f"{row['active_components']}/{row['original_components']}",
                        'Compression': f"{row['compression_ratio']:.1f}x",
                        'R²_Test': f"{row['r2_test']:.4f}",
                        'MSE_Test': f"{row['mse_test']:.2f}",
                        'R²_Loss': f"{r2_loss:.1f}%",
                        'MSE_Increase': f"{mse_increase:.1f}%"
                    })

        if summary_rows:
            summary_df = pd.DataFrame(summary_rows)
            print(summary_df.to_string(index=False))

In [ ]:
print("ENSEMBLE PRUNING ABLATION STUDY")
print("Random Forest: Rank-Based Pruning | XGBoost: Cumulative Pruning")
print("="*100)

# Initialize ensemble pruning study
ensemble_study = EnsemblePruningAblationStudy()

# Define pruning ratios
pruning_ratios_ensemble = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

X_train_arr = X_train_enc.values
y_train_arr = y_train.values
X_val_arr = X_val_enc.values
y_val_arr = y_val.values
X_test_arr = X_test_enc.values
y_test_arr = y_test.values

# Random Forest pruning study
if 'rf_model' in globals() or 'rf_model' in locals():
    print("Found Random Forest model - Running Rank-Based pruning study...")
    ensemble_study.run_ensemble_pruning_study(
        model=rf_model,
        model_name='Random Forest',
        X_train=X_train_arr,
        y_train=y_train_arr,
        X_test=X_test_arr,
        y_test=y_test_arr,
        X_val=X_val_arr,
        y_val=y_val_arr,
        pruning_ratios=pruning_ratios_ensemble
    )
else:
    print("Random Forest model not found. Please ensure 'rf_model' is available.")

# XGBoost pruning study
if 'xgb_model' in globals() or 'xgb_model' in locals():
    print("Found XGBoost model - Running Cumulative pruning study...")
    ensemble_study.run_ensemble_pruning_study(
        model=xgb_model,
        model_name='XGBoost',
        X_train=X_train_arr,
        y_train=y_train_arr,
        X_test=X_test_arr,
        y_test=y_test_arr,
        X_val=X_val_arr,
        y_val=y_val_arr,
        pruning_ratios=pruning_ratios_ensemble
    )
else:
    print("XGBoost model not found. Please ensure 'xgb_model' is available.")

# Plot and report
ensemble_study.plot_ensemble_pruning_results()
ensemble_study.generate_ensemble_pruning_report()

# Save results
ensemble_results_df = pd.DataFrame(ensemble_study.ensemble_results)
print(f"\nEnsemble pruning results saved with {len(ensemble_results_df)} entries")
print("\nPruning results:")
display(ensemble_results_df)


# Ablation Study Comparation

In [ ]:
# Combined Analysis: Neural Networks vs Ensemble Methods
def compare_all_pruning_methods():
    """
    Compare pruning effectiveness across all model types
    """
    print("\n" + "="*120)
    print("COMPREHENSIVE PRUNING COMPARISON: NEURAL NETWORKS vs ENSEMBLE METHODS")
    print("="*120)

    # Collect data from both studies
    all_models_comparison = []

    # Neural network results (if available)
    for _, result in pruning_results_df.iterrows():
        if result['pruning_ratio'] in [0.0, 0.3, 0.5, 0.7, 0.9]:
            all_models_comparison.append({
                'Model': result['model_name'],
                'Type': 'Neural Network',
                'Pruning_Method': 'L1 Norm',
                'Pruning_Ratio': result['pruning_ratio'],
                'R²_Test': result['r2_test'],
                'MSE_Test': result['mse_test'],
                'MAPE_Test': result['mape_test'],
                'MAE_Test': result['mae_test'],
                'Compression': result['compression_ratio'],
                'Components': f"{result['active_params']}/{result['original_params']}"
            })

    # Ensemble results
    for _, result in ensemble_results_df.iterrows():
        if result['pruning_ratio'] in [0.0, 0.3, 0.5, 0.7, 0.9]:
            pruning_method = 'Rank-Based' if result['model_name'] == 'Random Forest' else 'Cumulative'
            all_models_comparison.append({
                'Model': result['model_name'],
                'Type': 'Ensemble',
                'Pruning_Method': pruning_method,
                'Pruning_Ratio': result['pruning_ratio'],
                'R²_Test': result['r2_test'],
                'MSE_Test': result['mse_test'],
                'MAPE_Test': result['mape_test'],
                'MAE_Test': result['mae_test'],
                'Compression': result['compression_ratio'],
                'Components': f"{result['active_components']}/{result['original_components']}"
            })

    if all_models_comparison:
        comparison_df = pd.DataFrame(all_models_comparison)

        # Create pivot table for better visualization
        pivot_r2 = comparison_df.pivot_table(
            values='R²_Test',
            index=['Model', 'Type', 'Pruning_Method'],
            columns='Pruning_Ratio',
            fill_value=np.nan
        )
        pivot_mse = comparison_df.pivot_table(
            values='MSE_Test',
            index=['Model', 'Type', 'Pruning_Method'],
            columns='Pruning_Ratio',
            fill_value=np.nan
        )
        pivot_mape = comparison_df.pivot_table(
            values='MAPE_Test',
            index=['Model', 'Type', 'Pruning_Method'],
            columns='Pruning_Ratio',
            fill_value=np.nan
        )
        pivot_mae = comparison_df.pivot_table(
            values='MAE_Test',
            index=['Model', 'Type', 'Pruning_Method'],
            columns='Pruning_Ratio',
            fill_value=np.nan
        )

        print("\nR² Performance Across Pruning Levels:")
        print(pivot_r2.round(4))

        print("\nMSE Performance Across Pruning Levels:")
        print(pivot_mse.round(4))

        print("\nMAPE Performance Across Pruning Levels:")
        print(pivot_mape.round(4))

        print("\nMAE Performance Across Pruning Levels:")
        print(pivot_mae.round(4))

        pruning_levels = [0.3, 0.5, 0.7, 0.9]

        print(f"\n{'='*120}")
        print("PERFORMANCE RETENTION AT DIFFERENT PRUNING LEVELS")
        print("="*120)

        for pruning_level in pruning_levels:
            print(f"\n{'-'*60}")
            print(f"PERFORMANCE RETENTION AT {int(pruning_level*100)}% PRUNING")
            print(f"{'-'*60}")

            retention_summary = []
            for model in comparison_df['Model'].unique():
                model_data = comparison_df[comparison_df['Model'] == model]
                baseline = model_data[model_data['Pruning_Ratio'] == 0.0]
                pruned = model_data[model_data['Pruning_Ratio'] == pruning_level]

                if len(baseline) > 0 and len(pruned) > 0:
                    baseline_r2 = baseline.iloc[0]['R²_Test']
                    pruned_r2 = pruned.iloc[0]['R²_Test']
                    retention = pruned_r2 / baseline_r2 if baseline_r2 != 0 else 0

                    retention_summary.append({
                        'Model': model,
                        'Type': baseline.iloc[0]['Type'],
                        'Method': baseline.iloc[0]['Pruning_Method'],
                        'Baseline_R²': baseline_r2,
                        'Pruned_R²': pruned_r2,
                        'Retention': retention,
                        'Compression': pruned.iloc[0]['Compression']
                    })

            if retention_summary:
                retention_df = pd.DataFrame(retention_summary).sort_values('Retention', ascending=False)
                print(retention_df.round(4))

            best_model = retention_df.iloc[0]
            print(f"\nBEST PRUNING METHOD AT {int(pruning_level*100)}% LEVEL:")
            print(f"Model: {best_model['Model']} ({best_model['Type']})")
            print(f"Method: {best_model['Method']}")
            print(f"Performance Retention: {best_model['Retention']:.1%}")
            print(f"Compression Achieved: {best_model['Compression']:.1f}x")

    else:
        print("No pruning results available for comparison.")

# Run comprehensive comparison
compare_all_pruning_methods()

print("\n" + "="*120)
print("ABLATION STUDY COMPLETE")